In [8]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT license.



from IPython.display import display, Markdown, clear_output, HTML, Javascript, display_html
import subprocess
import sys
# display(Markdown(""" ## Checking required packages:
# #### if any are missing, installing them now..... """))
# !{sys.executable} -m pip install --no-cache-dir -r requirements_SP.txt
# clear_output() 


display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+1)'))

HTML('''<script>{
    $('div.input').hide();
    }''')

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(), IPython.notebook.get_selected_index()+1)'))
HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    The raw code for this IPython notebook is by default hidden for easier reading toggle on/off the raw code by clicking <a href="javascript:code_toggle()">here</a>.''')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [83]:

display(Markdown("##  File Validations"))
display(Markdown("Run this notebook once you have prepared all of the required input files - distance, space_capacity, team_size, and interactions files. This notebook validates your input files and prepares them in a Final Files folder. This is a required step before you can create a seating plan in the Generate Floorplan notebook."))

import pandas as pd
import random
import numpy as np
# import pulp
import csv
from io import StringIO
import os
import itertools
from IPython.display import HTML, Javascript, display_html,clear_output,Markdown
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from itertools import combinations
import scipy as sc
from scipy import optimize as op
from itertools import combinations
from datetime import datetime, date, time
import re
# display(Markdown('First in a series to validate input files:'))

# display(Markdown('* Team Size'))
# display(Markdown('* Interactions'))
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


##  File Validations

Run this notebook once you have prepared all of the required input files - distance, space_capacity, team_size, and interactions files. This notebook validates your input files and prepares them in a Final Files folder. This is a required step before you can create a seating plan in the Generate Floorplan notebook.

<IPython.core.display.Javascript object>

In [84]:


def createInteractionsMatrix2(flatInteractions):
    #get unique names of participants
    organizersNames = flatInteractions.iloc[:,0].unique()
    contributorsNames = flatInteractions.iloc[:,1].unique()
    allNames = set(organizersNames).union(set(contributorsNames))
    removeNames = ["Other_Collaborators", "Unclassified_Internal", "Unclassified_External", "Collaborators Within Group"]
    keepNames = allNames - set(removeNames)
    cols = flatInteractions.columns

    #add up withing group collaboration 
    temp = flatInteractions.copy()
    temp.loc[temp[cols[1]]=='Collaborators Within Group', cols[1]] = temp[cols[0]]
    temp = temp.drop(temp.columns[[2]], axis=1) # drop date column

    #remove unwanted rows
    temp =  temp[~ ((temp[cols[0]].isin(list(removeNames))) | (temp[cols[1]].isin(list(removeNames))))]

    #build matrix
    key_cols = [cols[0], cols[1]]
    interactionsMatrix = temp.set_index(key_cols).groupby(key_cols)[cols[-1]].agg(['sum']).unstack()


    #clean and wrap up
    df = interactionsMatrix
    df.columns = df.columns.droplevel(0)
    df.columns.name = None
    df.index.name = None
    df = df.reset_index()
    df.rename(columns = {df.columns[0]: 'Collab' }, inplace = True)
    return df

            
            
            
def sortInteractions(iDF):
    rawCols = iDF.columns
    idxMap = {} 
    #sort rows
    sortedRawI = iDF.sort_values([rawCols[0]])
    #keep old rows idx i.e. just put it aside and forget about it (kept copy for audit purposes)
    copyIdxRows = sortedRawI.reset_index().iloc[:, [0,1]]
    copyIdxRows.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["rows"] = copyIdxRows.copy()
    sortedRawI = sortedRawI.reset_index(drop = True)
    #transpose 
    temp = sortedRawI.set_index(sortedRawI.columns[0])
    transposeRawI = temp.transpose()
    transposeRawI = transposeRawI.reset_index()
    transposeRawI.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawI.columns.name = None
    convert_col_collab = transposeRawI.columns[0] 
    transposeRawI[convert_col_collab] = transposeRawI[convert_col_collab].astype(str)
    #sort columns 
    sortedRawI2 = transposeRawI.sort_values([transposeRawI.columns[0]])
    #record old column idx, put it aside and forget about it (just in case it is needed... kept a copy...)
    copyIdxCols = sortedRawI2.reset_index().iloc[:, [0,1]]
    copyIdxCols.rename(columns = {"index" : "orig_index"}, inplace = True)
    idxMap["cols"] = copyIdxCols.copy()
    sortedRawI2 = sortedRawI2.reset_index(drop = True)
    #transpose back
    temp2 = sortedRawI2.set_index(sortedRawI2.columns[0])
    transposeRawI2 = temp2.transpose()
    transposeRawI2 = transposeRawI2.reset_index()
    transposeRawI2.rename(columns = {"index" : rawCols[0]}, inplace = True)
    transposeRawI2.columns.name = None
    return transposeRawI2, idxMap 

def sortTeamsDF(tsDF, dropIdx = True):
    cols = tsDF.columns
    sortedDF = tsDF.sort_values(by = [cols[0]])
    if (dropIdx):
        sortedDF = sortedDF.reset_index(drop = True)
    else:
        sortedDF = sortedDF.reset_index()
    return sortedDF

def getTeamIdsDF(sRawTS):
    return (sRawTS[[sRawTS.columns[0]]].reset_index())[[sRawTS.columns[0],"index"]]

            
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
            
    

<IPython.core.display.Javascript object>

In [85]:
loadfiles_markdown = widgets.Output()

with loadfiles_markdown:
    display(Markdown('<div class="alert alert-block alert-info"><b> Load Data: </b> Select your file(s).</div>'))

display(loadfiles_markdown)

        
cwd = os.getcwd()
tab_nest_001 = widgets.Accordion()
style = {'description_width': '185px'}
layout = {'width': '505px'}

def go_to_nextblock(btn):
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+3)'))


input_path_int = FileChooser(cwd)
input_path_ts = FileChooser(cwd)
input_path_dist = FileChooser(cwd)
input_path_fc = FileChooser(cwd)

#outfilename= widgets.Text(value=None,description='out_filename')
go_run_full = widgets.Button(description='Go',button_style = 'primary')




tab_nest_001.children = [input_path_int,input_path_ts,input_path_fc,input_path_dist]

tab_nest_001.set_title(0,'Interactions')
tab_nest_001.set_title(1,'Team size')
tab_nest_001.set_title(2,'Space Capacity')
tab_nest_001.set_title(3,'Distance')


display(widgets.VBox([tab_nest_001,go_run_full]))
    
go_run_full.on_click(go_to_nextblock)     

<IPython.core.display.Javascript object>

Output()

In [86]:
int_input_path = input_path_int.selected_path + '\\'+ input_path_int.selected_filename
ts_input_path = input_path_ts.selected_path + '\\'+ input_path_ts.selected_filename

fc_input_path = input_path_fc.selected_path + '\\'+ input_path_fc.selected_filename
dist_input_path = input_path_dist.selected_path + '\\'+ input_path_dist.selected_filename

    
#out_filename = outfilename.value

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
CGREEN  = '\33[32m'
CRED = '\033[91m'
CEND = '\033[0m'
BOLD = '\033[1m'

flatInteractionBool = True
rawIraw = pd.read_csv(int_input_path)

rawIraw_old = rawIraw.copy()

out = widgets.Output(layout={'border': '1px solid black'})

# Keep 1st 4 cols and  remove remaining empty columns 
rawIraw = rawIraw.drop(rawIraw.columns[4:], axis=1)

rawIraw = (rawIraw.replace(r'^\s*$', np.nan, regex=True))
rawIraw = rawIraw.dropna(axis=0,how='all')
rawIraw = rawIraw.dropna(axis = 1, how = 'all')
# Replace Scenario 1a NaNs in CB hrs to 0.
rawIraw['Collaboration_hours'] = rawIraw['Collaboration_hours'].replace(np.nan,0)



rawTS = pd.read_csv(ts_input_path)

# Keep 1st 2 cols (TS) and  remove remaining empty columns 
rawTS  = rawTS.drop(rawTS.columns[2:], axis=1)
rawTS  = rawTS.replace(r'^\s*$', np.nan, regex=True)


# Keep 1st 2 cols (FC) and  remove remaining empty columns and get total # of rows for Distance file.

rawFC = pd.read_csv(fc_input_path)
rawFC  = rawFC.drop(rawFC.columns[2:], axis=1)

rawFC_reduced = rawFC.copy()
rawFC_reduced = rawFC_reduced.dropna()
rawFC_reduced   = rawFC_reduced.replace(r'^\s*$', np.nan, regex=True)
no_of_floors  = len(rawFC_reduced)



rawDT = pd.read_csv(dist_input_path)
rawDT  = rawDT.drop(rawDT.columns[no_of_floors+1:], axis=1)
rawDT  = rawDT.replace(r'^\s*$', np.nan, regex=True)



scenario1a_markdown = widgets.Output()

with scenario1a_markdown:
    display(Markdown('<div class="alert alert-block alert-warning"><b> Scenario 1a: </b> Although this should not happen. In case; All teams present, but NaN collaboration, replace NaN collaboration with 0.</div>'))

display(scenario1a_markdown)
display(Markdown("Number of rows within your <b>Interaction file</b> that contain *NaN* for Collaboration_hours:"))

    
if rawIraw_old['Collaboration_hours'].isnull().sum() > 0:
    display(HTML('&#10060; Found ' + '<b>'+str(rawIraw_old['Collaboration_hours'].isnull().sum())+'</b>' + ' NaNs '))
    display(out)
else:
    display(HTML('<p>&#9989; Passed</p>'))

    print('Collaboration_hours       ' +str(rawIraw_old['Collaboration_hours'].isnull().sum()))
    display(out)
        
    
    
    

    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


<IPython.core.display.Javascript object>

In [ ]:
scenario1b_markdown = widgets.Output()

with scenario1b_markdown:
    display(Markdown('<div class="alert alert-block alert-warning"><b> Scenario 1b: </b> Scrub any whitespaces/empty rows for all 4 files.<b> No action required from user</b>. This will be auto-fixed if any found..</div>'))

blankrawI = rawIraw_old.isnull().sum()
blankrawFC = rawFC.isnull().sum()
blankrawDT = rawDT.isnull().sum()
blankrawTS = rawTS.isnull().sum()

pass_or_fail_mkdwn0 = widgets.Output()
showscn1b = widgets.Button(description ='Show details')
hidescn1b = widgets.Button(description ='Hide details')

scn1b_mkdwn = widgets.Output()
with scn1b_mkdwn:
    display(Markdown("Number of rows per column that contain *empty rows* or *white-space* for <b>all files</b>:"))

with pass_or_fail_mkdwn0:
    clear_output()
    if blankrawI.sum() > 0 or blankrawDT.sum() > 0 or blankrawFC.sum() > 0 or blankrawTS.sum() > 0:
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))
display(widgets.VBox([widgets.VBox([scenario1b_markdown,widgets.HBox([pass_or_fail_mkdwn0,scn1b_mkdwn])]),showscn1b]))
# display(out)




def hide_details_scn1b_details(click):
    clear_output()
    display(widgets.VBox([widgets.VBox([scenario1b_markdown,widgets.HBox([pass_or_fail_mkdwn0,scn1b_mkdwn])]),showscn1b]))
    display(out)

def click_on_show_scn1b_details(click):
    clear_output()
    display(widgets.VBox([widgets.VBox([scenario1b_markdown,widgets.HBox([pass_or_fail_mkdwn0,scn1b_mkdwn])]),hidescn1b]))
    display(widgets.HTML(value="<ins><b>Interaction file</b></ins>"))
    if blankrawI.sum() > 0:
        display(HTML('&#10060; Found  ' +'<b>'+ str(blankrawI.sum())+'</b>' + ' cells that contain nulls or white spaces. '))
    else:
        display(HTML('<p>&#9989; Passed</p>'))
    print('--------------------------------')
    print(blankrawI)

    display(widgets.HTML(value="<ins><b>Team size file</b></ins>"))
    if blankrawTS.sum() > 0:
        display(HTML('&#10060; Found  ' +'<b>'+ str(blankrawTS.sum())+'</b>' + ' cells that contain nulls or white spaces. '))
        display(Markdown(" Any NaN or blank values inside size column will be replaced with 0."))
    else:
        display(HTML('<p>&#9989; Passed</p>'))
    print('--------------------------------')
    print(blankrawTS)


    display(widgets.HTML(value="<ins><b>Floor capacity file</b></ins>"))
    if blankrawFC.sum() > 0:
        display(HTML('&#10060; Found  ' +'<b>'+ str(blankrawFC.sum())+'</b>' + ' cells that contain nulls or white spaces. '))
    else:
        display(HTML('<p>&#9989; Passed</p>'))
    print('--------------------------------')
    print(blankrawFC)


    display(widgets.HTML(value="<ins><b>Distance file</b></ins>"))
    if blankrawDT.sum() > 0:
        display(HTML('&#10060; Found  ' +'<b>'+ str(blankrawDT.sum())+'</b>' + ' cells that contain nulls or white spaces. '))
    else:
        display(HTML('<p>&#9989; Passed</p>'))
    print('--------------------------------')
    print(blankrawDT)
    display(out)





    








showscn1b.on_click(click_on_show_scn1b_details)
hidescn1b.on_click(hide_details_scn1b_details)


display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


<IPython.core.display.Javascript object>

In [ ]:
clear_output()
#display(rawI.head())
# Convert TI and CB to string columns for Raw interactions file:
Col_1 = rawIraw.columns[0]
Col_2 = rawIraw.columns[1]

rawIraw[Col_1] = rawIraw[Col_1].apply(str)
rawIraw[Col_2] = rawIraw[Col_2].apply(str)

# Scenario 1b: Get rid of whitespaces by replacing spaces with NaNs for all 4 files
# Remove/drop  replaced NaNs from 4 files
# rawIraw = (rawIraw.replace(r'^\s*$', np.nan, regex=True))
# rawIraw = rawIraw.dropna(axis=0,how='all')
# rawIraw = rawIraw.dropna(axis='columns')
# # Replace Scenario 1a NaNs in CB hrs to 0.
# rawIraw['Collaboration_hours'] = rawIraw['Collaboration_hours'].replace(np.nan,0)

col1_TS = rawTS.columns[0]

rawTS = (rawTS.replace(r'^\s*$', np.nan, regex=True))
rawTS = rawTS.dropna(axis=0,how='all')
rawTS = rawTS.dropna(axis = 1, how = 'all')
rawTS[col1_TS] = rawTS[col1_TS].astype(str)
rawFC = (rawFC.replace(r'^\s*$', np.nan, regex=True))
rawFC = rawFC.dropna(axis=0,how='all')
rawFC = rawFC.dropna(axis = 1, how = 'all')
rawDT = (rawDT.replace(r'^\s*$', np.nan, regex=True))
rawDT = rawDT.dropna(axis=0,how='all')
rawDT = rawDT.dropna(axis = 1, how = 'all')

if(flatInteractionBool):
    rawI = createInteractionsMatrix2(rawIraw)

sRawTS = sortTeamsDF(rawTS)
teamsIds = getTeamIdsDF(sRawTS)
teamLabels = dict([ [id, team] for team, id in teamsIds.to_dict(orient = 'split')['data']])
#display(sRawTS.head())
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))


In [ ]:

teamsize_mkdown = widgets.Output()
teamsize_mkdown2 = widgets.Output()

outreq = widgets.Output(layout={'border': '2px solid blue'})

tab_nest_002 = widgets.Accordion(selected_index=None)
style = {'description_width': '185px'}
layout = {'width': '505px'}




with teamsize_mkdown:
    display(Markdown('<div class="alert alert-block alert-info"><b> Step 1. Validate Team Size:</div>'))
#display(teamsize_mkdown)

with teamsize_mkdown2:

    display(Markdown('<p>&nbsp; &nbsp; a. <b>Validate columns layout – different per mode</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Standard-should have 2 columns and match fixed schema ( "Team" and "Size"):</p>'))
    display(Markdown('<p>&emsp; &emsp; ii.Co-located Teams: n columns (participants in column names should match)</p>'))
    display(Markdown('<p>&nbsp; &nbsp; b. <b>Get unique name of participants</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Ensure they are unique (no duplicates)</p>'))
    #display(Markdown('<p>&emsp; &emsp; ii. Check no empty or missing names</p>'))
    display(Markdown('<p>&nbsp; &nbsp; c. <b>Validate Values</b>.</p>'))
#     display(Markdown('<p>&emsp; &emsp; i. No NA, None or string</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Value > 0.Should be positive integer. </p>'))
    display(Markdown("<p>&emsp; &emsp; ii. Team names should not contain invalid characters. Invalid chars: *|,:<>[]{}`';@&$#%</p>"))
    display(Markdown('<p>&emsp; &emsp; iii. Numeric </p>'))
    display(Markdown('<p>&nbsp; &nbsp; d. <b>Calculate total team size</b>.</p>'))
    display(Markdown('<p>&nbsp; &nbsp; e. <b>If Co-located Teams</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Check name of participants match</p>'))
    display(Markdown('<p>&emsp; &emsp; ii. Check Adjacency seats have valid numeric values </p>'))
    
    

tab_nest_002.children = [widgets.VBox([teamsize_mkdown,teamsize_mkdown2])]

tab_nest_002.set_title(0,'Validate Team Size File Requirements')

display(tab_nest_002)
display(outreq)

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    

In [ ]:


clear_output()

# 1.a Validate columns layout – different per mode.
#        i.	Standard/Optimize: 2 columns
showtsz_colslayout = widgets.Button(description ='Show details')
hidetsz_colslayout = widgets.Button(description ='Hide details')
tszvalidatecols_mkdn = widgets.Output()
pass_or_fail_mkdwn1 = widgets.Output()


if len(rawTS.columns) >=2 :
    tszColheader1= rawTS.columns[0]
    tszColheader2= rawTS.columns[1]
else:
    ""


with tszvalidatecols_mkdn:
    display(Markdown('<b>Validate columns layout</b>'))
display(tszvalidatecols_mkdn)
with pass_or_fail_mkdwn1:
    clear_output()
    if len(rawTS.columns) == 2 and (tszColheader1 == 'Team' and tszColheader2 == 'Size'):
        display(HTML('<p>&#9989;'))
    else:
        display(HTML('&#10060;'))
        
        
        
display(widgets.HBox([pass_or_fail_mkdwn1,showtsz_colslayout]))
display(out)


tsz_schema_error_markdown = widgets.Output()

with tsz_schema_error_markdown:
    if (len(rawTS.columns)>=2):
        if (tszColheader1 == 'Team' and tszColheader2 == 'Size'):
            ""
        elif (tszColheader1 != 'Team' and tszColheader2 == 'Size'):
            clear_output()
            display(Markdown('<div class="alert alert-block alert-danger"><b> ERROR! Invalid column headers for Standard/Optimize: </b> '+ tszColheader1 + ' was found instead of expected schema of "Team".</div>'))
            print("Please fix this error in your Team Size file and re-upload and click Go again.")
        elif (tszColheader1 == 'Team' and tszColheader2 != 'Size'):
            clear_output()
            display(Markdown('<div class="alert alert-block alert-danger"><b> ERROR! Invalid column headers for Standard/Optimize: </b> '+ tszColheader2 + ' was found instead of expected schema of "Size".</div>'))
            print("Please fix this error in your Team Size file and re-upload and click Go again.")
        elif (tszColheader1 != 'Team' and tszColheader2 != 'Size'):
            clear_output()
            display(Markdown('<div class="alert alert-block alert-danger"><b> ERROR! Invalid column headers for Standard/Optimize: </b> '+ tszColheader1 + ' was found instead of expected schema of "Team" and ' + tszColheader2 + ' was found instead of expected schema of "Size".</div>'))
            print("Please fix this error in your Team Size file and re-upload and click Go again.")
    else:
        clear_output()
        display(Markdown('<div class="alert alert-block alert-danger"><b> ERROR! Invalid number of columns for Standard: </b> Team Size file should contain 2 columns with the following schema. ( "Team" and "Size".</div>'))
        print("Please fix this error in your Team Size file and re-upload and click Go again.")



def hide_details_tsz_cols_details(click):
    clear_output()
    display(tszvalidatecols_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn1,showtsz_colslayout]))
    display(tsz_schema_error_markdown)
    display(out)

def click_on_show_tsz_cols_details(click):
    clear_output()
    display(tszvalidatecols_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn1,hidetsz_colslayout]))
    display(tsz_schema_error_markdown)
    print(BOLD,"Number of Columns:",CEND, len(rawTS.columns))
    print(BOLD,"Column headers: ",CEND, list(rawTS.columns))
    if (len(rawTS.columns)>=2):
        if len(rawTS.columns) == 2:
            print(CGREEN +u'\u2713'+CEND, BOLD,"Valid number of columns for Standard/Optimize",CEND)
        if (tszColheader1 == 'Team' and tszColheader2 == 'Size'):
            print(CGREEN +u'\u2713'+CEND, BOLD,"Valid column headers for Standard/Optimize",CEND)
        elif (tszColheader1 != 'Team' and tszColheader2 == 'Size'):
            print(CRED +u'\u2717'+CEND, BOLD,"Invalid column headers for Standard/Optimize",CEND)
            print(tszColheader1 + " was found instead of expected schema of 'Team'")
        elif (tszColheader1 == 'Team' and tszColheader2 != 'Size'):
            print(CRED +u'\u2717'+CEND, BOLD,"Invalid column headers for Standard/Optimize",CEND)
            print(tszColheader2 + " was found instead of expected schema of 'Size'")
        elif (tszColheader1 != 'Team' and tszColheader2 != 'Size'):
            print(CRED +u'\u2717'+CEND, BOLD,"Invalid column headers for Standard/Optimize",CEND)
            print(tszColheader1 + " was found instead of expected schema of 'Team'")
            print(tszColheader2 + " was found instead of expected schema of 'Size'")
        
 

        
    display(out)
    

if (len(rawTS.columns)>=2) and (tszColheader1 == 'Team' and tszColheader2 == 'Size'):
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
else:
    None

sRawI, idxMapI = sortInteractions(rawI)
showtsz_colslayout.on_click(click_on_show_tsz_cols_details)
hidetsz_colslayout.on_click(hide_details_tsz_cols_details)
display(tsz_schema_error_markdown)
#     sRawI.head()
#display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [ ]:

clear_output()
# 1b. Get unique name of participants
#     i.	Ensure they are unique (no duplicates)
teamsTSraw = rawTS.iloc[:,[0]]
tszCol1= rawTS.columns[0]

#find duplicates
temp = teamsTSraw
temp["cnt"] = 1
temp = temp.groupby([tszCol1], as_index = False) ["cnt"].sum().sort_values(by = ["cnt"])
duplicateTeams = temp[temp.cnt > 1]
nonDuplicateTeams = temp[temp.cnt == 1]


pass_or_fail_mkdwn2 = widgets.Output()
showtsz_uniqueptps = widgets.Button(description ='Show details')
hidetsz_uniqueptps = widgets.Button(description ='Hide details')
tszuniquenames_mkdn = widgets.Output()
with tszuniquenames_mkdn:
    display(Markdown('<b>Get unique name of participants</b>'))
display(tszuniquenames_mkdn)
with pass_or_fail_mkdwn2:
    clear_output()
    if len(duplicateTeams)> 0:
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))
display(widgets.HBox([pass_or_fail_mkdwn2,showtsz_uniqueptps]))
display(out)





def hide_details_tsz_unique_ptps(click):
    clear_output()
    display(tszuniquenames_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn2,showtsz_uniqueptps]))
    display(out)




def click_on_show_tsz_unique_ptps(click):
    clear_output()
    display(tszuniquenames_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn2,hidetsz_uniqueptps]))
    if len(duplicateTeams)> 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Number of duplicated teams:",CEND, len(duplicateTeams))
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"Number of duplicated teams:",CEND, len(duplicateTeams))
    print(BOLD,"Number of unique teams:",CEND, len(nonDuplicateTeams))
    display(out)

teamCatalog = nonDuplicateTeams[[tszCol1]].copy()
teamCatalog = teamCatalog.sort_values(by = tszCol1).reset_index(drop = True)
teamCatalog[tszCol1] = teamCatalog[tszCol1].astype(str)
teamCatalog.head()


showtsz_uniqueptps.on_click(click_on_show_tsz_unique_ptps)
hidetsz_uniqueptps.on_click(hide_details_tsz_unique_ptps)

tempI = np.asmatrix(sRawI.iloc[0:, 1:])
pd.DataFrame(tempI)
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+3)'))

<IPython.core.display.Javascript object>

In [ ]:

# clear_output()

# showtsz_missingnames = widgets.Button(description ='Show details')
# hidetsz_missingnames = widgets.Button(description ='Hide details')
# tszmissingnames_mkdn = widgets.Output()
# with tszmissingnames_mkdn:
#     display(Markdown('<b>Check for empty or missing names</b>'))
# display(tszmissingnames_mkdn)
# display(showtsz_missingnames)


# # 1b. Get unique name of participants
# #    ii.	Check no empty or missing names
# df = rawTS.copy()
# isnullDF = df.replace(r'^\s*$', np.nan, regex=True).isnull()
#numMissingTeams = isnullDF.iloc[:,0].sum()

# def hide_details_tsz_missingnames(click):
#     clear_output()
#     display(tszmissingnames_mkdn)
#     display(showtsz_missingnames)

# def click_on_show_tsz_missingnames(click):
#     clear_output()
#     display(tszmissingnames_mkdn)
#     display(hidetsz_missingnames)
#     print("num(empty/blank)Teams Names: ", numMissingTeams)
#     if(numMissingTeams>0):
#         errorMessage = "Team name is mandatory. \"Team\" column cannot be empty or blank." 
#         emptyRowList = (np.where(isnullDF.iloc[:,0])[0]).tolist()
    
    
# # display(Markdown('### Step 2. Detect missing values'))
# #Get unique name of Teams
# teamsOnCol1 = set(rawIraw[rawIraw.columns[0]])
# teamsOnCol2 = set(rawIraw[rawIraw.columns[1]])
# allNames = set(teamsOnCol1).union(set(teamsOnCol2))
# teamCatalog2 = pd.DataFrame(list(allNames))
# #print("Number of Raw Team Names in Col 1: ", len(teamsOnCol1))
# #print("Number of Raw Team Names in Col 2: ", len(teamsOnCol2))
# #print("Number or Raw Team Names in Both Cols: ", len(allNames))
    
    

# showtsz_missingnames.on_click(click_on_show_tsz_missingnames)
# hidetsz_missingnames.on_click(hide_details_tsz_missingnames)


# display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [ ]:

clear_output()
# c.	Validate Values
 #       i.	No NA, None or string
 #       ii.	Numeric
 #       iii.	Value > 0
df = rawTS.copy()    

excludeKeywords = ["@","#","$","%","&","*","+","=","|",":",";","<",">",",",".","/","[","]","{","}","\\"]
checkinvalidTS = rawTS.copy()
team_Col = checkinvalidTS.columns[0]

checkinvalidTS[team_Col] = checkinvalidTS[team_Col].apply(str)
invalidTS = checkinvalidTS[checkinvalidTS.apply(lambda r: any([kw in r[0] for kw in excludeKeywords]), axis=1)]

if len(rawTS.columns) == 2:
    teamSizeValuesDF = df.iloc[:,[1]]
    teamSizeValuesDF = teamSizeValuesDF.fillna(0)
    numInvalidSize = (teamSizeValuesDF.iloc[:,0].astype(int) < 0).sum()
else:
    teamSizeValuesDF = 0
    numInvalidSize = 0


showtsz_validatevalues = widgets.Button(description ='Show details')
hidetsz_validatevalues = widgets.Button(description ='Hide details')
pass_or_fail_mkdwn3 = widgets.Output()
tszvv_mkdn = widgets.Output()

with pass_or_fail_mkdwn3:
    clear_output()
    if(numInvalidSize>0) or len(invalidTS) > 0:
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))
        

with tszvv_mkdn:
    display(Markdown('<b>Validate values</b>'))
display(tszvv_mkdn)
# if(numInvalidSize>0) or len(invalidTS) > 0:
#     display(HTML('&#10060;'))
# else:
#     display(HTML('<p>&#9989;'))
display(widgets.HBox([pass_or_fail_mkdwn3,showtsz_validatevalues])) 
display(out)


    
def hide_details_tsz_vv(click):
    clear_output()
    display(tszvv_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn3,showtsz_validatevalues])) 
    display(out)

invalidSizeRows = teamSizeValuesDF[teamSizeValuesDF.iloc[:,0].astype(int)<0] 


def click_on_show_tsz_vv(click):
    clear_output()
    display(tszvv_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn3,hidetsz_validatevalues]))
    display(Markdown(""" #### i:""")) 
    if(numInvalidSize>0):
        print(CRED+u'\u2717'+CEND,BOLD,"Team size cannot be negative",CEND)
        print(invalidSizeRows)
        print("-------------------------------------------------------------")
    else:
        print(CGREEN +u'\u2713'+CEND, BOLD,"numInvalidSize Values (Non-positive intergers):",CEND, numInvalidSize )
        print("-------------------------------------------------------------")
    if len(invalidTS) > 0:
        display(Markdown(""" #### ii:"""))
        print(CRED+u'\u2717'+CEND,BOLD,"Below are the following Team names that may contain invalid characters",CEND)
        display(invalidTS['Team'])
    display(out)
        
#replaceValue = 0
showtsz_validatevalues.on_click(click_on_show_tsz_vv)
hidetsz_validatevalues.on_click(hide_details_tsz_vv)

#print("As requested, replacing all missing values by default value: ", replaceValue)

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [ ]:

# clear_output()
# from datetime import date

# today = date.today()
showtsz_totalseats = widgets.Button(description ='Show details')
hidetsz_totalseats = widgets.Button(description ='Hide details')
tsztotal_mkdn = widgets.Output()
with tsztotal_mkdn:
    display(Markdown('<b>Total team size</b>'))
display(tsztotal_mkdn)

pass_or_fail_mkdwn4 = widgets.Output()
with pass_or_fail_mkdwn4:
    clear_output()
    display(HTML('<p>&#9989;'))

display(widgets.HBox([pass_or_fail_mkdwn4,showtsz_totalseats]))

display(out)
    
def hide_details_tsz_totalseats(click):
    clear_output()
    display(tsztotal_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn4,showtsz_totalseats]))
    display(out)

# Calculate total team size
rawTS = rawTS.fillna(0)
if len(rawTS.columns) == 2:
    totalSeatsRequested = sum(rawTS.iloc[:,1].astype(int))
else:
    totalSeatsRequested = 0

def click_on_show_tsz_totalseats(click):
    clear_output()
    display(tsztotal_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn4,hidetsz_totalseats]))
    print(BOLD,"Total Seats Requested:",CEND, totalSeatsRequested)
    display(out)

    
    
showtsz_totalseats.on_click(click_on_show_tsz_totalseats)
hidetsz_totalseats.on_click(hide_details_tsz_totalseats)    
# mvCols = list(rawIraw.columns[0:2])
# amountCol = rawIraw.columns[3]
# NaNs = np.argwhere(np.isnan(tempI))
# missingValuesDF = pd.DataFrame(NaNs, columns = mvCols)
# missingValuesDF["Date"] = today.strftime("%m/%d/%y")
# missingValuesDF[amountCol] = "None"
# missingValuesDF["Comment"] = 'Missing Value'
# missingValuesDF["Replace by?"] = replaceValue
# #print("Number of Values Missing: ", len(missingValuesDF))
# #missingValuesDF.head()
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [ ]:
clear_output()
#display(Markdown('### Step 1. Validate Team Size File'))

tab_nest_003 = widgets.Accordion(selected_index=None)
style = {'description_width': '185px'}
layout = {'width': '505px'}
    



spccap_mkdown = widgets.Output()

with spccap_mkdown:
    display(Markdown('<div class="alert alert-block alert-info"><b> Step 2. Validate Space Capacity File:</div>'))
#display(intx_mkdown)


spccap_mkdown2 = widgets.Output()

with spccap_mkdown2:
    display(Markdown('<p>&nbsp; &nbsp; a. <b>Validate columns layout – different per mode</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Standard-should have 2 columns and match fixed schema ( "Floor" and "Total Capacity"):</p>'))
    display(Markdown('<p>&nbsp; &nbsp; b. <b>Validate Spaces Catalog</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Space names - Ensure they are unique (no duplicates)</p>'))
    #display(Markdown('<p>&emsp; &emsp; ii. Check no empty or missing names</p>'))
    display(Markdown('<p>&nbsp; &nbsp; c. <b>Validate Values</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Number of seats available - should be positive integer</p>'))
    display(Markdown("<p>&emsp; &emsp; ii. Space names should not contain invalid characters. Invalid chars: *|,:<>[]{}`';@&$#%</p>"))
    display(Markdown('<p>&emsp; &emsp; iii. The sum of team sizes should be less than or equal to the sum of the space capacities</p>'))
    
tab_nest_003.children = [widgets.VBox([spccap_mkdown,spccap_mkdown2])]

tab_nest_003.set_title(0,'Validate Space File Requirements')

display(tab_nest_003)
display(outreq)
#K = len(teamCatalog2)
#     print("Number of Teams: ", K)
#     print("Total Team Combinations: ", K*K)
#     print("Number of rows in Interactions File: ", len(rawIraw) )
#     print("Missing + Existing: ", len(rawIraw) + len(missingValuesDF))
#     print("Number of missing rows: ", len(missingValuesDF))
#     print("Ratio of mising rows: ", len(missingValuesDF) / len(rawIraw))
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    

In [ ]:
clear_output()

# 1.a Validate columns layout – different per mode.
#        i.	Standard/Optimize: 2 columns
showspc_colslayout = widgets.Button(description ='Show details')
hidespc_colslayout = widgets.Button(description ='Hide details')
spcvalidatecols_mkdn = widgets.Output()
pass_or_fail_mkdwn5 = widgets.Output()

if  len(rawFC.columns) >= 2:
    spacesColheader1= rawFC.columns[0]
    spacesColheader2= rawFC.columns[1]
else:
    spacesColheader1= rawFC.columns[0]
    


with spcvalidatecols_mkdn:
    display(Markdown('<b>Validate columns layout</b>'))
display(spcvalidatecols_mkdn)
with pass_or_fail_mkdwn5:
    clear_output()
    if len(rawFC.columns) == 2 and (spacesColheader1 == 'Floor' and spacesColheader2 == 'Total Capacity'):
        display(HTML('<p>&#9989;'))
    else:
        display(HTML('&#10060;'))
        
display(widgets.HBox([pass_or_fail_mkdwn5,showspc_colslayout]))
display(out)


spc_schema_error_markdown = widgets.Output()

with spc_schema_error_markdown:
    if (len(rawFC.columns) >= 2):
        if  (spacesColheader1 == 'Floor' and spacesColheader2 == 'Total Capacity'):
            ""
        elif (spacesColheader1 != 'Floor' and spacesColheader2 == 'Total Capacity'):
            clear_output()
            display(Markdown('<div class="alert alert-block alert-danger"><b> ERROR! Invalid column headers for Standard/Optimize: </b> '+ spacesColheader1 + ' was found instead of expected schema of "Floor".</div>'))
            print("Please fix this error in your Space Capacity file and re-upload and click Go again.")
        elif  (spacesColheader1 == 'Floor' and spacesColheader2 != 'Total Capacity'):
            clear_output()
            display(Markdown('<div class="alert alert-block alert-danger"><b> ERROR! Invalid column headers for Standard/Optimize: </b> '+ spacesColheader2 + ' was found instead of expected schema of "Total Capacity".</div>'))
            print("Please fix this error in your Space Capacity file and re-upload and click Go again.")
        elif (spacesColheader1 != 'Floor' and spacesColheader2 != 'Total Capacity'):
            clear_output()
            display(Markdown('<div class="alert alert-block alert-danger"><b> ERROR! Invalid column headers for Standard/Optimize: </b> '+ spacesColheader1 + ' was found instead of expected schema of "Floor" and ' + spacesColheader2 + ' was found instead of expected schema of "Total Capacity".</div>'))
            print("Please fix this error in your Space Capacity file and re-upload and click Go again.")
    else:
        clear_output()
        display(Markdown('<div class="alert alert-block alert-danger"><b> ERROR! Invalid number of columns for Standard: </b> Space Capacity file should contain 2 columns with the following schema. ( "Floor" and "Total Capacity".</div>'))
        print("Please fix this error in your Space Capacity file and re-upload and click Go again.")




def hide_details_spc_cols_details(click):
    clear_output()
    display(spcvalidatecols_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn5,showspc_colslayout]))
    display(spc_schema_error_markdown)
    display(out)

def click_on_show_spc_cols_details(click):
    clear_output()
    display(spcvalidatecols_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn5,hidespc_colslayout]))
    display(spc_schema_error_markdown)
    
    print(BOLD,"Number of Columns:",CEND, len(rawFC.columns))
    print(BOLD,"Column headers: ",CEND, list(rawFC.columns))
    if len(rawFC.columns) == 2:
        print(CGREEN + u'\u2713'+CEND,BOLD,"Valid number of columns for Standard",CEND)
    if len(rawFC.columns) >= 2:
        if (spacesColheader1 == 'Floor' and spacesColheader2 == 'Total Capacity'):
            print(CGREEN +u'\u2713'+CEND, BOLD,"Valid column headers for Standard/Optimize",CEND)
        elif (spacesColheader1 != 'Floor'  and spacesColheader2 == 'Total Capacity'):
            print(CRED +u'\u2717'+CEND, BOLD,"Invalid column headers for Standard/Optimize",CEND)
            print(spacesColheader1  + " was found instead of expected schema of 'Floor'")
        elif (spacesColheader1  == 'Floor' and spacesColheader2 != 'Total Capacity'):
            print(CRED +u'\u2717'+CEND, BOLD,"Invalid column headers for Standard/Optimize",CEND)
            print(spacesColheader2 + " was found instead of expected schema of 'Total Capacity'")
        elif (spacesColheader1  != 'Floor' and spacesColheader2 != 'Total Capacity'):
            print(CRED +u'\u2717'+CEND, BOLD,"Invalid column headers for Standard/Optimize",CEND)
            print(spacesColheader1 + " was found instead of expected schema of 'Floor'")
            print(spacesColheader2 + " was found instead of expected schema of 'Total Capacity'")
        
    display(out)

showspc_colslayout.on_click(click_on_show_spc_cols_details)
hidespc_colslayout.on_click(hide_details_spc_cols_details)
display(spc_schema_error_markdown)
#     sRawI.head()
if (len(rawFC.columns) >= 2) and (spacesColheader1 == 'Floor' and spacesColheader2 == 'Total Capacity'):
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
else:
    None
#display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [ ]:
clear_output()
# 1b. Get unique name of participants
#     i.	Ensure they are unique (no duplicates)
spacesFCraw = rawFC.iloc[:,[0]]
spacesCol1= rawFC.columns[0]

#find duplicates
tempspc = spacesFCraw
tempspc["cnt"] = 1
tempspc = tempspc.groupby([spacesCol1], as_index = False) ["cnt"].sum().sort_values(by = ["cnt"])
duplicateSpaces = tempspc[tempspc.cnt > 1]
nonDuplicateSpaces = tempspc[tempspc.cnt == 1]


pass_or_fail_mkdwn6 = widgets.Output()
showspc_unique = widgets.Button(description ='Show details')
hidespc_unique = widgets.Button(description ='Hide details')
spcuniquenames_mkdn = widgets.Output()
with spcuniquenames_mkdn:
    display(Markdown('<b>Get unique name of Spaces</b>'))
display(spcuniquenames_mkdn)
with pass_or_fail_mkdwn6:
    clear_output()
    if len(duplicateSpaces)== 0:
        display(HTML('<p>&#9989;'))
    else:
        display(HTML('&#10060;'))
        
display(widgets.HBox([pass_or_fail_mkdwn6,showspc_unique]))
display(out)





def hide_details_spc_unique_spaces(click):
    clear_output()
    display(spcuniquenames_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn6,showspc_unique]))
    display(out)




def click_on_show_spc_unique_spaces(click):
    clear_output()
    display(spcuniquenames_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn6,hidespc_unique]))
    
    if len(duplicateSpaces)> 0:
        print(CRED + u'\u2717'+CEND,BOLD,"Number of duplicated spaces:",CEND, len(duplicateSpaces))
        print(duplicateSpaces)
        print('----------------------------------------')
    else:
        print(CGREEN + u'\u2713'+CEND,BOLD,"Number of duplicated spaces:",CEND, len(duplicateSpaces))
    print(BOLD,"Number of unique spaces:",CEND, len(nonDuplicateSpaces))
    display(out)

teamCatalogSC = nonDuplicateSpaces[[spacesCol1]].copy()
teamCatalogSC = teamCatalogSC.sort_values(by = spacesCol1).reset_index(drop = True)
teamCatalogSC[spacesCol1] = teamCatalogSC[spacesCol1].astype(str)
teamCatalogSC.head()


showspc_unique.on_click(click_on_show_spc_unique_spaces)
hidespc_unique.on_click(hide_details_spc_unique_spaces)

    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+3)'))

<IPython.core.display.Javascript object>

In [ ]:

clear_output()
# c.	Validate Values
 #       i.	second column should be a positive integer (number of seats available)
 #       ii.	No invalid characters
 #       iii.	The sum of team sizes should be less than or equal to the sum of the floor capacities



dfSPCagg = rawFC.copy()
dfSPCagg = dfSPCagg.fillna(0)

if len(rawFC.columns) == 2:
    grptotalSPC = dfSPCagg.iloc[:,1].astype(int).sum()
else:
    grptotalSPC = 0


# excludeKeywords = ["@","#","$","%","&","*","_","+","-","=","|",":",";","<",">",",",".","/","(",")","[","]","{","}","\\"]
checkinvalidSP = rawFC.copy()
space_Col = checkinvalidSP.columns[0]

checkinvalidSP[space_Col] = checkinvalidSP[space_Col].apply(str)
invalidSP = checkinvalidSP[checkinvalidSP.apply(lambda r: any([kw in r[0] for kw in excludeKeywords]), axis=1)]

if len(rawFC.columns) == 2:
    spaceSeatsValuesDF = dfSPCagg.iloc[:,[1]]
    spaceSeatsValuesDF = spaceSeatsValuesDF.fillna(0)
    numInvalidSizeSP = (spaceSeatsValuesDF.iloc[:,0].astype(int) < 0).sum()
else:
    spaceSeatsValuesDF = 0
    numInvalidSizeSP = 0
    


if totalSeatsRequested <= grptotalSPC:
    filecomp_TS_SPC = 0
else:
    filecomp_TS_SPC = 1

showspc_validatevalues = widgets.Button(description ='Show details')
hidespc_validatevalues = widgets.Button(description ='Hide details')
pass_or_fail_mkdwn7 = widgets.Output()
spcvv_mkdn = widgets.Output()

with pass_or_fail_mkdwn7:
    clear_output()
    if(numInvalidSizeSP>0) or len(invalidSP) > 0 or filecomp_TS_SPC == 1:
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))
        

with spcvv_mkdn:
    display(Markdown('<b>Validate values</b>'))
display(spcvv_mkdn)
# if(numInvalidSize>0) or len(invalidTS) > 0:
#     display(HTML('&#10060;'))
# else:
#     display(HTML('<p>&#9989;'))
display(widgets.HBox([pass_or_fail_mkdwn7,showspc_validatevalues])) 
display(out)


    
def hide_details_spc_vv(click):
    clear_output()
    display(spcvv_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn7,showspc_validatevalues])) 
    display(out)

    


def click_on_show_spc_vv(click):
    clear_output()
    display(spcvv_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn7,hidespc_validatevalues]))
    
    display(Markdown(""" #### i:""")) 
    if(numInvalidSizeSP>0):
        print(CRED+u'\u2717'+CEND,BOLD,"Space capacity cannot be negative",CEND)
        print("-------------------------------------------------------------")
        print(spaceSeatsValuesDF[spaceSeatsValuesDF.iloc[:,0]<0])
    else:
        print(CGREEN + u'\u2713'+CEND, BOLD,"numInvalidSize Values (Non-positive intergers):",CEND, numInvalidSizeSP)
        print("-------------------------------------------------------------")
    if len(invalidSP) > 0:
        display(Markdown(""" #### ii:"""))
        print(CRED+ u'\u2717'+CEND, BOLD,"Below are the following space names that may contain invalid characters:",CEND)
        display(invalidSP[space_Col])
        print("-------------------------------------------------------------")

    if filecomp_TS_SPC == 1:
        display(Markdown(""" #### iii:"""))
        print(CRED+ u'\u2717'+CEND, BOLD,"The sum of team sizes is not less than or equal to the sum of the space capacities.",CEND)
        print(str(totalSeatsRequested)+"(Team-size)" +" > " + str(grptotalSPC)+"(Space-capacity)" )
        print("-------------------------------------------------------------")
    else:
        display(Markdown(""" #### iii:"""))
        print(CGREEN + u'\u2713'+CEND,BOLD,"Team size total is less than equal to space capacity total:",CEND,str(totalSeatsRequested)+"(Team-size)" +" <= " + str(grptotalSPC)+"(Space-capacity)" )
        
    display(out)
        
#replaceValue = 0
showspc_validatevalues.on_click(click_on_show_spc_vv)
hidespc_validatevalues.on_click(hide_details_spc_vv)

#print("As requested, replacing all missing values by default value: ", replaceValue)

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [ ]:
clear_output()
#display(Markdown('### Step 1. Validate Team Size File'))

tab_nest_004 = widgets.Accordion(selected_index=None)
style = {'description_width': '185px'}
layout = {'width': '505px'}
    



dist_mkdown = widgets.Output()

with dist_mkdown:
    display(Markdown('<div class="alert alert-block alert-info"><b> Step 3. Validate Distance File:</div>'))
#display(intx_mkdown)


dist_mkdown2 = widgets.Output()

with dist_mkdown2:
    display(Markdown('<p>&nbsp; &nbsp; a. <b>Validate columns layout – different per mode</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Columns and rows should be equal in number:</p>'))
    display(Markdown('<p>&emsp; &emsp; ii. Needs to be a N * N matrix. Where N is the number of Spaces in Space Capacity File:</p>'))
    display(Markdown('<p>&emsp; &emsp; iii. Columns and rows should have the same names in the same order:</p>'))
    display(Markdown('<p>&nbsp; &nbsp; b. <b>Validate Space Catalog - in rows</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Row names should be unique (no duplicates)</p>'))
    display(Markdown('<p>&emsp; &emsp; ii. Get unique name of spaces in Rows</p>'))
    display(Markdown('<p>&nbsp; &nbsp; c. <b>Validate Space Catalog - in columns</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Column names should be unique (no duplicates)</p>'))
    display(Markdown('<p>&emsp; &emsp; ii. Get unique name of spaces in Columns</p>'))
    display(Markdown('<p>&nbsp; &nbsp; d. <b>Validate Space Catalog - Columns vs Rows</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. (<b>For Space Capacity File</b>): Names in Space Capacity and not in Distance Matrix --> user needs to add these spaces to distance matrix</p>'))
    display(Markdown('<p>&emsp; &emsp; ii. (<b>For Distance File</b>): Any space names in Rows and not in Columns --> user needs to add these space names to the Columns</p>'))
    display(Markdown('<p>&emsp; &emsp; iii. (<b>For Distance File</b>): Any space names in Columns and not in Rows --> user needs to add these space names to the Rows</p>'))
    display(Markdown('<p>&emsp; &emsp; iv. (<b>For Distance File</b>): Any space name in Rows and not in Space Capacity (or viceversa) --> user needs to add these space names to Space Capacity file</p>'))
    display(Markdown('<p>&emsp; &emsp; v. (<b>For Distance File</b>): Any space name in Columns and not in Space Capacity (or viceversa) --> user needs to add these space names to Space Capacity file</p>'))
    display(Markdown('<p>&nbsp; &nbsp; e. <b>Validate Values</b>.</p>'))
    display(Markdown('<p>&emsp; &emsp; i. Matrix values should be positive numeric values (both positive decimals and positive integers are valid for Distance file values)</p>'))
    
tab_nest_004.children = [widgets.VBox([dist_mkdown,dist_mkdown2])]

tab_nest_004.set_title(0,'Validate Distance File Requirements')

display(tab_nest_004)
display(outreq)
#K = len(teamCatalog2)
#     print("Number of Teams: ", K)
#     print("Total Team Combinations: ", K*K)
#     print("Number of rows in Interactions File: ", len(rawIraw) )
#     print("Missing + Existing: ", len(rawIraw) + len(missingValuesDF))
#     print("Number of missing rows: ", len(missingValuesDF))
#     print("Ratio of mising rows: ", len(missingValuesDF) / len(rawIraw))
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    

<IPython.core.display.Javascript object>

In [ ]:
clear_output()



# 1.a Validate columns layout – different per mode.
#        i.	Standard/Optimize: 2 columns

rem1colDT = rawDT.copy()
rem1colDT.drop(rem1colDT.columns[0],axis=1,inplace=True)
distHeaders = list(set(rem1colDT.columns))
diststripHeaders = [x.strip(' ') for x in distHeaders]
# diststripHeaders.remove("Unnamed: 0")


#2.a.	Validate columns layout
#        i.	Needs to be an N*N matrix 
#        ii.	N is the number of Spaces – From Space Capacity File
N = teamCatalogSC.shape[0]
teamCatalogSetDT = set(teamCatalogSC.iloc[:,0])
tempDT = rawDT.copy()
SpacesOnColsDT = set(tempDT.columns[1:])
SpacesOnRowsDT = set(tempDT.iloc[:,0])
if(len(SpacesOnColsDT) == len(SpacesOnRowsDT)):
    diff1DT = set()
elif(len(SpacesOnColsDT) > len(SpacesOnRowsDT)):
    diff1DT = SpacesOnColsDT - SpacesOnRowsDT
else:
    diff1DT = SpacesOnRowsDT - SpacesOnColsDT    
if(N == len(SpacesOnColsDT)):
    diff2DT = set()
elif(N > len(SpacesOnColsDT)):
    diff2DT = teamCatalogSetDT - SpacesOnColsDT 
else:    
    diff2DT = SpacesOnColsDT - teamCatalogSetDT
if(N == len(SpacesOnRowsDT)):
    diff3DT = set()
elif(N > len(SpacesOnRowsDT)):
    diff3DT = teamCatalogSetDT - SpacesOnRowsDT
else:    
    diff3DT = SpacesOnRowsDT - teamCatalogSetDT

#3.a.	Validate columns layout
#     columns and rows should have the same names in order.

headerscheckDT = list(rem1colDT)
headerscheckDT2 = [x.strip(' ') for x in headerscheckDT]
#headerscheckDT2.remove("Unnamed: 0")

distrowHeader = rawDT.columns[0]
rowscheckDT = list(rawDT[distrowHeader])





showdist_colslayout = widgets.Button(description ='Show details')
hidedist_colslayout = widgets.Button(description ='Hide details')
distvalidatecols_mkdn = widgets.Output()
pass_or_fail_mkdwn8 = widgets.Output()

with distvalidatecols_mkdn:
    display(Markdown('<b>Validate Distance Matrix columns layout</b>'))
display(distvalidatecols_mkdn)
with pass_or_fail_mkdwn8:
    clear_output()
    if len(diststripHeaders) == len(rawDT) :
        display(HTML('<p>&#9989;'))
    elif ((N != len(SpacesOnColsDT) or N != len(SpacesOnRowsDT)) or len(diststripHeaders) != len(rawDT)):
        display(HTML('&#10060;'))
        
display(widgets.HBox([pass_or_fail_mkdwn8,showdist_colslayout]))
display(out)



def hide_details_dist_cols_details(click):
    clear_output()
    display(distvalidatecols_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn8,showdist_colslayout]))
    display(out)

def click_on_show_dist_cols_details(click):
    clear_output()
    display(distvalidatecols_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn8,hidedist_colslayout]))
    
    display(Markdown(""" #### i:""")) 
    if len(diststripHeaders) == len(rawDT):
        print(CGREEN + u'\u2713'+CEND,BOLD,"Valid number of columns and rows for Standard",CEND)
        print("-------------------------------------------------------------")
        print("Number of Columns:", len(diststripHeaders))
        print("Number of Rows: ", len(rawDT))
    else:
        print(CRED + u'\u2717'+CEND,BOLD,"Invalid number of columns and rows for Standard",CEND)
        print("-------------------------------------------------------------")
        print("Number of Columns:", len(diststripHeaders))
        print("Number of Rows: ", len(rawDT))
        
    display(Markdown(""" #### ii:""")) 
    if(N != len(SpacesOnColsDT) or N != len(SpacesOnRowsDT)):
        print(CRED + u'\u2717'+CEND,BOLD,"Distance Matrix needs to be a N*N matrix",CEND)
        print("-------------------------------------------------------------")
    else:
        print(CGREEN + u'\u2713'+CEND,BOLD,"Distance Matrix needs to be a N*N matrix",CEND)
        print("-------------------------------------------------------------")
    print("Number of Unique Spaces in Space Capacity File: ", N)
    print("Number of Unique Spaces in Cols: ", len(SpacesOnColsDT))
    print("Number of Unique Spaces in Rows: ", len(SpacesOnRowsDT))
    
    

    if(N != len(SpacesOnColsDT) or N != len(SpacesOnRowsDT)):
        print(CRED,"**File Validation Error**",CEND,": Distance Matrix needs to be a N*N matrix")
    if(diff1DT):
        print("Difference between Columns and Rows in Distance File...")
        print(diff1DT)
    elif(diff2DT):
        print("Difference between Space Capacity and columns in Distance File...")
        print(diff2DT)
    elif(diff3DT):
        print("Difference between Space Capacity and rows in Distance File...")
        print(diff3DT)
        
    display(Markdown(""" #### iii:"""))
    if headerscheckDT2 == rowscheckDT:
        print(CGREEN + u'\u2713'+CEND,BOLD,"Columns and rows have the same names in same order",CEND)
        
    else:
        print(CRED + u'\u2717'+CEND,BOLD,"Columns and rows have the same names in same order",CEND)
        print("-------------------------------------------------------------")
        print(BOLD," Names in columns: ",CEND,headerscheckDT2)
        display(Markdown("""#### <p>&emsp; &emsp; vs. </p>"""))
        print(BOLD," Names in rows:",CEND ,rowscheckDT)
        
    
     
    display(out)

showdist_colslayout.on_click(click_on_show_dist_cols_details)
hidedist_colslayout.on_click(hide_details_dist_cols_details)
#     sRawI.head()
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [ ]:

clear_output()
# b. Get unique name of space catalogs-in rows
#     i.	Ensure they are unique (no duplicates)
distSpaceraw = rawDT.iloc[:,[0]].copy()
distspcCol1 = rawDT.columns[0]
#find duplicates
tempdtspc = distSpaceraw
tempdtspc["cnt"] = 1
tempdtspc = tempdtspc.groupby([distspcCol1], as_index = False) ["cnt"].sum().sort_values(by = ["cnt"])
duplicatedistSpcs = tempdtspc[tempdtspc.cnt > 1]
nonDuplicatedistSpcs = tempdtspc[tempdtspc.cnt == 1]



showdist_spccatalog = widgets.Button(description ='Show details')
hidedist_spccatalog = widgets.Button(description ='Hide details')
distvalidatespc_mkdn = widgets.Output()
pass_or_fail_mkdwn9 = widgets.Output()

with distvalidatespc_mkdn:
    display(Markdown('<b>Validate Distance Space Catalog - in rows</b>'))
display(distvalidatespc_mkdn)
with pass_or_fail_mkdwn9:
    clear_output()
    if len(duplicatedistSpcs)> 0 :
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))
        
display(widgets.HBox([pass_or_fail_mkdwn9,showdist_spccatalog]))
display(out)

def hide_details_dist_spcs_details(click):
    clear_output()
    display(distvalidatespc_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn9,showdist_spccatalog]))
    display(out)

def click_on_show_dist_spcs_details(click):
    clear_output()
    display(distvalidatespc_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn9,hidedist_spccatalog]))
    
    if len(duplicatedistSpcs)> 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Number of duplicated spaces:",CEND, len(duplicatedistSpcs))
        
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"Number of duplicated spaces:",CEND, len(duplicatedistSpcs))
        
    print(BOLD,"Number of unique spaces:",CEND, len(nonDuplicatedistSpcs))
    print("-------------------------------------------------------------")
    
    
    print(BOLD,f'Unique Space Names-Rows:',CEND,f'\n{nonDuplicatedistSpcs}')
    
    display(out)
    
    
showdist_spccatalog.on_click(click_on_show_dist_spcs_details)
hidedist_spccatalog.on_click(hide_details_dist_spcs_details)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [ ]:

clear_output()
# b. Get unique name of space catalogs-in columns
#     ii.	Ensure they are unique (no duplicates)


showdist_spccatalog2 = widgets.Button(description ='Show details')
hidedist_spccatalog2 = widgets.Button(description ='Hide details')
distvalidatespc_mkdn2 = widgets.Output()
pass_or_fail_mkdwn10 = widgets.Output()

duplicatedistSpcs2 = len(headerscheckDT2)-len(set(headerscheckDT2))


with distvalidatespc_mkdn2:
    display(Markdown('<b>Validate Distance Space Catalog - in columns</b>'))
display(distvalidatespc_mkdn2)
with pass_or_fail_mkdwn10:
    clear_output()
    if len(headerscheckDT2) != len(set(headerscheckDT2)) :
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))
        
display(widgets.HBox([pass_or_fail_mkdwn10,showdist_spccatalog2]))
display(out)

def hide_details_dist_spcs_details2(click):
    clear_output()
    display(distvalidatespc_mkdn2)
    display(widgets.HBox([pass_or_fail_mkdwn10,showdist_spccatalog2]))
    display(out)

def click_on_show_dist_spcs_details2(click):
    clear_output()
    display(distvalidatespc_mkdn2)
    display(widgets.HBox([pass_or_fail_mkdwn10,hidedist_spccatalog2]))
    
    if duplicatedistSpcs2 > 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Number of duplicated spaces:",CEND, duplicatedistSpcs2)
        
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"Number of duplicated spaces:",CEND, duplicatedistSpcs2)
        
    print(BOLD,"Number of unique spaces:",CEND, len(set(headerscheckDT2)))
    print("-------------------------------------------------------------")
    
    
    print(BOLD,'Unique Space Names-Cols:',CEND,headerscheckDT2)
    
    display(out)
    
    
showdist_spccatalog2.on_click(click_on_show_dist_spcs_details2)
hidedist_spccatalog2.on_click(hide_details_dist_spcs_details2)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [ ]:
clear_output()
# d. Get unique name of space catalogs-in columns
#     i.	space capacity, names in space capacity and not in distance matrix--> User needs to add these spaces to distance matrix

spcnames_nondup = list(set(spacesFCraw[spacesCol1]))
distspcnames_nondup = list(nonDuplicatedistSpcs.iloc[:,0])
namesin_dist_not_spc = [i for i in distspcnames_nondup if i not in spcnames_nondup]
namesin_spc_not_dist=[i for i in spcnames_nondup if i not in distspcnames_nondup]
namesin_distrows_not_spc=[i for i in distspcnames_nondup if i not in spcnames_nondup]
namesin_distcols_not_spc=[i for i in headerscheckDT2 if i not in spcnames_nondup]

#    ii.	space capacity, names in in distance matrix and not Team Size--> User needs to add these spaces to Team size file.
#tsznames_nondup = list(nonDuplicateTeams.iloc[:,0])
anynames_in_dist= list(set(headerscheckDT2) | set(distspcnames_nondup))
#namesin_dist_not_tsz =[i for i in anynames_in_dist if i not in tsznames_nondup]

namesin_distrows_not_distcols=[i for i in distspcnames_nondup if i not in headerscheckDT2]

namesin_distcols_not_distrows=[i for i in headerscheckDT2 if i not in distspcnames_nondup]

showdist_spcvsdist = widgets.Button(description ='Show details')
hidedist_spcvsdist = widgets.Button(description ='Hide details')
distspcvsdist_mkdn = widgets.Output()
pass_or_fail_mkdwn11 = widgets.Output()


with distspcvsdist_mkdn:
    display(Markdown('<b>Validate Distance Space Catalog - Columns vs Rows</b>'))
display(distspcvsdist_mkdn)
with pass_or_fail_mkdwn11:
    clear_output()
    if ((len(namesin_spc_not_dist)> 0 or len(namesin_dist_not_spc) >0) or len(namesin_distrows_not_distcols) >0):
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))
        
display(widgets.HBox([pass_or_fail_mkdwn11,showdist_spcvsdist]))
display(out)



def hide_details_spcvsdist_details(click):
    clear_output()
    display(distspcvsdist_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn11,showdist_spcvsdist]))
    display(out)

def click_on_show_spcvsdist_details(click):
    clear_output()
    display(distspcvsdist_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn11,hidedist_spcvsdist]))
    
    display(Markdown(""" #### i: For Space Capacity File - Space Capacity vs. Distance"""))
    if len(namesin_spc_not_dist) > 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Names in space capacity file and not in distance matrix :",CEND,namesin_spc_not_dist)
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"Names in space capacity file and not in distance matrix :",CEND,None)
    if len(namesin_dist_not_spc) > 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Names in distance matrix and not in space capacity file :",CEND,namesin_dist_not_spc)
        print("-------------------------------------------------------------")
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"Names in distance matrix and not in space capacity file:",CEND,None)
        print("-------------------------------------------------------------")
    
#     display(Markdown(""" #### ii: For Space Capacity File - Team size vs. Distance"""))
#     if len(namesin_dist_not_tsz) > 0:
#         print(CRED+u'\u2717'+CEND,BOLD,"Names in distance matrix and not in team size file :",CEND,namesin_dist_not_tsz)
#     else:
#         print(CGREEN+u'\u2713'+CEND,BOLD,"Names in distance matrix and not in team size file :",CEND,None)
#         print("-------------------------------------------------------------")
        
    display(Markdown(""" #### ii: For Distance File - Names in rows and not in columns"""))
    if len(namesin_distrows_not_distcols) > 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Names in distance rows and not in distance columns :",CEND,namesin_distrows_not_distcols)
        print("-------------------------------------------------------------")
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"Names in distance rows and not in distance columns :",CEND,None)
        print("-------------------------------------------------------------")
        
        
    display(Markdown(""" #### iii: For Distance File - Names in columns and not in rows"""))
    if len(namesin_distcols_not_distrows) > 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Names in distance columns and not in distance rows :",CEND,namesin_distcols_not_distrows)
        print("-------------------------------------------------------------")
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"Names in distance columns and not in distance rows :",CEND,None)
        print("-------------------------------------------------------------")
        
        
        
    display(Markdown(""" #### iv: For Distance File - Any space name in rows and not in Space Capacity"""))
    if len(namesin_distrows_not_spc) > 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Names in distance rows and not in space capacity file :",CEND,namesin_distrows_not_spc)
        print("-------------------------------------------------------------")
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"Names in distance rows and not in space capacity file :",CEND,None)
        print("-------------------------------------------------------------")
    
    display(Markdown(""" #### v: For Distance File - Any space name in columns and not in Space Capacity"""))
    if len(namesin_distcols_not_spc) > 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Names in distance columns and not in space capacity file :",CEND,namesin_distcols_not_spc)
        print("-------------------------------------------------------------")
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"Names in distance columns and not in space capacity file :",CEND,None)
        print("-------------------------------------------------------------")
     
    

    
        
    
        
        
showdist_spcvsdist.on_click(click_on_show_spcvsdist_details)
hidedist_spcvsdist.on_click(hide_details_spcvsdist_details)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [ ]:
clear_output()
# b. Get unique name of space catalogs-in columns
#     ii.	Ensure they are unique (no duplicates)


showdist_vv = widgets.Button(description ='Show details')
hidedist_vv = widgets.Button(description ='Hide details')
distvalidatevalues_mkdn = widgets.Output()
pass_or_fail_mkdwn12 = widgets.Output()

psttestDT = rawDT.copy()
psttestDT2 = psttestDT.drop(psttestDT.columns[0], axis = 1).apply(pd.to_numeric)
numInvalidSizeDT = len(psttestDT2.columns[(psttestDT2 < 0).any()])
colInvalidSizeDT = psttestDT2.columns[(psttestDT2 < 0).any()]

with distvalidatevalues_mkdn:
    display(Markdown('<b>Validate Distance Values</b>'))
display(distvalidatevalues_mkdn)
with pass_or_fail_mkdwn12:
    clear_output()
    if numInvalidSizeDT > 0 :
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))
        
display(widgets.HBox([pass_or_fail_mkdwn12,showdist_vv]))
display(out)


def hide_details_distvv_details(click):
    clear_output()
    display(distvalidatevalues_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn12,showdist_vv]))
    display(out)

def click_on_show_distvv_details(click):
    clear_output()
    display(distvalidatevalues_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn12,hidedist_vv]))
    
    display(Markdown(""" #### i: Validate Distance Values - Should be positive integer """))
    if numInvalidSizeDT > 0:
        print(CRED+u'\u2717'+CEND,BOLD,"Matrix values should be positive numeric values for distance file :",CEND, "The following columns contain non-positive values : ", colInvalidSizeDT)
    else:
        print(CGREEN+u'\u2713'+CEND,BOLD,"All matrix values are positive numeric for distance file:",CEND)
        
        
showdist_vv.on_click(click_on_show_distvv_details)
hidedist_vv.on_click(hide_details_distvv_details)
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [ ]:
clear_output()
#display(Markdown('### Step 1. Validate Team Size File'))
tab_nest_005 = widgets.Accordion(selected_index=None)

intx_mkdown = widgets.Output()

with intx_mkdown:
    display(Markdown('<div class="alert alert-block alert-info"><b> Step 4. Validate Interactions File:</div>'))

    display(Markdown('<p>&nbsp; &nbsp; a. <b>Validate columns layout</b>.</p>'))
    display(Markdown('<p>&emsp; &nbsp; i. Needs to be an K*K matrix :</p>'))
    display(Markdown('<p>&emsp; &nbsp; ii. K is the number of Teams – From Team Size File</p>'))
    display(Markdown('<p>&nbsp; &nbsp; b. <b>Get unique name of Teams</b>.</p>'))
    display(Markdown('<p>&emsp; &nbsp; i. Check Time Investors Column</p>'))
    display(Markdown('<p>&emsp; &emsp; 1. Ensure they are unique (no duplicates)</p>'))
    display(Markdown('<p>&emsp; &emsp; 2. Check no empty or missing names</p>'))
    display(Markdown('<p>&emsp; &emsp; 3. Check WPA specific names: For Example</p>'))
    display(Markdown('           a. These are acceptable names but will be removed (not counted) ='))
    display(Markdown('              ["Other_Collaborators", "Unclassified_Internal", "Unclassified_External", "Collaborators Within Group"]'))
    display(Markdown('           b. "Collaborators Within Group" is acceptable in Collaborators column ONLY'))
    display(Markdown('           c. No need to worry about these names ->  Currently the createInteractionMatrix2 routine in python will drop them.'))
    display(Markdown('<p>&emsp; &nbsp; ii. Check Collaborators Column</p>'))
    display(Markdown('<p>&emsp; &emsp; 1. Ensure they are unique (no duplicates)</p>'))
    display(Markdown('<p>&emsp; &emsp; 2. Check no empty or missing names</p>'))
    display(Markdown('<p>&emsp; &emsp; 3. Check WPA specific names: For Example</p>'))
    display(Markdown('           a. These are acceptable names but will be removed (not counted) ='))
    display(Markdown('              ["Other_Collaborators", "Unclassified_Internal", "Unclassified_External", "Collaborators Within Group"]'))
    display(Markdown('           b. "Collaborators Within Group" is acceptable in Collaborators column ONLY'))
    display(Markdown('           c. No other WPA specific names are accepted (might be difficult to detect them)'))
    display(Markdown('           d. No need to worry about these names ->  Currently the createInteractionMatrix2 routine in python will drop them.'))
    display(Markdown('<p>&nbsp; &nbsp; c. <b>Ensure Time Collaborators + Time Investors columns names match</b>.</p>'))
    display(Markdown('<p>&emsp; &nbsp; i. With Team Size File</p>'))

    display(Markdown('<p>&nbsp; &nbsp; d. <b>Validate Values</b>.</p>'))
    display(Markdown('<p>&emsp; &nbsp; i. No NA, None or string</p>'))
    display(Markdown('<p>&emsp; &nbsp; ii. Numeric </p>'))
    display(Markdown('<p>&emsp; &nbsp; iii. Value > 0 </p>'))
    
tab_nest_005.children = [widgets.VBox([intx_mkdown])]

tab_nest_005.set_title(0,'Validate Interaction File Requirements')
display(tab_nest_005)
display(outreq)
#K = len(teamCatalog2)
#     print("Number of Teams: ", K)
#     print("Total Team Combinations: ", K*K)
#     print("Number of rows in Interactions File: ", len(rawIraw) )
#     print("Missing + Existing: ", len(rawIraw) + len(missingValuesDF))
#     print("Number of missing rows: ", len(missingValuesDF))
#     print("Ratio of mising rows: ", len(missingValuesDF) / len(rawIraw))
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
   

<IPython.core.display.Javascript object>

In [ ]:

clear_output()

#a.	Validate columns layout
#        i.	Needs to be an K*K matrix 
#        ii.	K is the number of Teams – From Team Size File
K = teamCatalog.shape[0]
teamCatalogSet = set(teamCatalog.iloc[:,0])
temp = createInteractionsMatrix2(rawIraw)
teamsOnCols = set(temp.columns[1:])
teamsOnRows = set(temp.iloc[:,0])
if(len(teamsOnCols) == len(teamsOnRows)):
    diff1 = set()
elif(len(teamsOnCols) > len(teamsOnRows)):
    diff1 = teamsOnCols - teamsOnRows
else:
    diff1 = teamsOnRows - teamsOnCols    
if(K == len(teamsOnCols)):
    diff2 = set()
elif(K > len(teamsOnCols)):
    diff2 = teamCatalogSet - teamsOnCols 
else:    
    diff2 = teamsOnCols - teamCatalogSet
if(K == len(teamsOnRows)):
    diff3 = set()
elif(K > len(teamsOnRows)):
    diff3 = teamCatalogSet - teamsOnRows
else:    
    diff3 = teamsOnRows - teamCatalogSet





showintx_cols_layout = widgets.Button(description ='Show details')
hideintx_cols_layout = widgets.Button(description ='Hide details')
intxcolsvalidate_mkdn = widgets.Output()
with intxcolsvalidate_mkdn:
    display(Markdown('<b>Validate Interaction Matrix columns layout</b>'))
display(intxcolsvalidate_mkdn)

pass_or_fail_mkdwn13 = widgets.Output()
with pass_or_fail_mkdwn13:
    clear_output()
    if(K != len(teamsOnCols) or K != len(teamsOnRows)):
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))

display(widgets.HBox([pass_or_fail_mkdwn13,showintx_cols_layout]))


# if(K != len(teamsOnCols) or K != len(teamsOnRows)):
#     display(HTML('&#10060;'))
# else:
#     display(HTML('<p>&#9989;'))
# display(showintx_cols_layout) 
display(out)


def hide_details_intx_colslayout(click):
    clear_output()
    display(intxcolsvalidate_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn13,showintx_cols_layout]))
    display(out)


    
    
def click_on_show_intx_colslayout(click):
    clear_output()
    display(intxcolsvalidate_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn13,hideintx_cols_layout]))
    print("Number of Unique Teams in Team Size File: ", K)
    print("Number of Unique Teams in Cols: ", len(teamsOnCols))
    print("Number of Unique Teams in Rows: ", len(teamsOnRows))
    


    if(K != len(teamsOnCols) or K != len(teamsOnRows)):
        print(CRED,"**File Validation Error**",CEND,": Interaction Matrix needs to be a K*K matrix")
    if(diff1):
        print("Difference between "+ Col_1 + " and " + Col_2 + " in Interactions File...")
        print(diff1)
    elif(diff2):
        print("Difference between Team Size and "+ Col_1 + " Interactions File...")
        print(diff2)
    display(out)

showintx_cols_layout.on_click(click_on_show_intx_colslayout)
hideintx_cols_layout.on_click(hide_details_intx_colslayout)

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [ ]:
clear_output()

ti_Col = rawIraw.columns[0]
cb_Col = rawIraw.columns[1]

excludeKeywords = ["@","#","$","%","&","*","_","+","-","=","|",":",";","<",">",",",".","/","(",")","[","]","{","}","\\"]
checkinvalidINTX = rawIraw.copy()

removeNames = ["Other_Collaborators", "Unclassified_Internal", "Unclassified_External", "Collaborators Within Group"]

checkinvalidINTX[ti_Col] = checkinvalidINTX[ti_Col].apply(str)
checkinvalidINTX[cb_Col] = checkinvalidINTX[cb_Col].apply(str)
invalidTI= checkinvalidINTX[checkinvalidINTX.apply(lambda r: any([kw in r[0] for kw in excludeKeywords]), axis=1)]

invalidTI = invalidTI[~invalidTI[ti_Col].isin(removeNames)]

invalidTI = invalidTI[[ti_Col]].drop_duplicates()

invalidCB= checkinvalidINTX[checkinvalidINTX.apply(lambda r: any([kw in r[1] for kw in excludeKeywords]), axis=1)]

invalidCB = invalidCB[~invalidCB[cb_Col].isin(removeNames)]

invalidCB = invalidCB[[cb_Col]].drop_duplicates()


showintx_ticb_invalid = widgets.Button(description ='Show details')
hideintx_ticb_invalid = widgets.Button(description ='Hide details')
intxinvalid_mkdn = widgets.Output()
with intxinvalid_mkdn:
    display(Markdown('<b> Checking invalid character values - Time Investors & Collaborators column</b>'))
display(intxinvalid_mkdn)

pass_or_fail_mkdwn14 = widgets.Output()
with pass_or_fail_mkdwn14:
    clear_output()
    if len(invalidTI) > 0 or len(invalidCB) >0:
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))

display(widgets.HBox([pass_or_fail_mkdwn14,showintx_ticb_invalid]))



# if len(invalidTI) > 0 or len(invalidCB) >0:
#     display(HTML('&#10060;'))
# else:
#     display(HTML('<p>&#9989;'))

# display(showintx_ticb_invalid) 
display(out)


def hide_details_intx_ticbinvalid(click):
    clear_output()
    display(intxinvalid_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn14,showintx_ticb_invalid]))
    display(out)



def click_on_show_intx_ticbinvalid(click):
    clear_output()
    display(intxinvalid_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn14,hideintx_ticb_invalid]))
    if len(invalidTI) > 0 and len(invalidCB) < 1:
        print("Invalid " + ti_Col + ": " + invalidTI .to_string(index=False,header=False))
    elif len(invalidTI) < 1 and len(invalidCB) > 0:
        print("Invalid " + cb_Col + ": " + invalidCB .to_string(index=False,header=False))
    elif len(invalidTI) > 0 and len(invalidCB) > 0:
        print("Invalid " + ti_Col + ": " + invalidTI .to_string(index=False,header=False))
        print("Invalid " + cb_Col + ": " + invalidCB .to_string(index=False,header=False))
    else:
        print("No invalid characters found in "+ ti_Col + " or " + cb_Col +".")
#     if(numMissingTeams>0):
#         errorMessage = "Team name is mandatory. \"Team\" column cannot be empty or blank." 
#         emptyRowList = (np.where(isnullDF.iloc[:,0])[0]).tolist()
    display(out)

        
showintx_ticb_invalid.on_click(click_on_show_intx_ticbinvalid)
hideintx_ticb_invalid.on_click(hide_details_intx_ticbinvalid)        
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+4)'))

<IPython.core.display.Javascript object>

In [ ]:

# clear_output()



# showintx_ti_missing = widgets.Button(description ='Show details')
# hideintx_ti_missing = widgets.Button(description ='Hide details')
# intxtimissing_mkdn = widgets.Output()
# with intxtimissing_mkdn:
#     display(Markdown('<b> Check missing values - Time Investors column</b>'))
# display(intxtimissing_mkdn)    
# display(showintx_ti_missing) 


# def hide_details_intx_timissing(click):
#     clear_output()
#     display(intxtimissing_mkdn)  
#     display(showintx_ti_missing)



# # display(Markdown('#####  Check no empty None or missing values - Time Investors Column'))

# #        i.	Check Time Investors Column
# #         2.	Check no empty or missing names
# #2.	Check no empty (None, N/A NaN or missing names) in Time Investors Columns
# cols = rawIraw.columns 
# timeInvestors = rawIraw[[rawIraw.columns[0]]].copy()
# df = timeInvestors.copy()
# isnullDF = df.replace(r'^\s*$', np.nan, regex=True).isnull()
# numMissingTeams = isnullDF.iloc[:,0].sum()


# def click_on_show_intx_timissing(click):
#     clear_output()
#     display(intxtimissing_mkdn)  
#     display(hideintx_ti_missing)
    
#     print("numMissingTeams Names: ", numMissingTeams)
#     if(numMissingTeams>0):
#         errorMessage = "Team name is mandatory. \"Team\" column cannot be empty or blank." 
#         emptyRowList = (np.where(isnullDF.iloc[:,0])[0]).tolist()

        
# showintx_ti_missing.on_click(click_on_show_intx_timissing)
# hideintx_ti_missing.on_click(hide_details_intx_timissing)        
    
# display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [ ]:

# clear_output()

# showintx_cb_missing = widgets.Button(description ='Show details')
# hideintx_cb_missing = widgets.Button(description ='Hide details')
# intxcbmissing_mkdn = widgets.Output()
# with intxcbmissing_mkdn:
#     display(Markdown('<b> Check missing values - Collaborators column</b>'))
# display(intxcbmissing_mkdn)
# display(showintx_cb_missing) 


# def hide_details_intx_cbmissing(click):
#     clear_output()
#     display(intxcbmissing_mkdn)
#     display(showintx_cb_missing)


# # display(Markdown('#####  Check no empty None or missing values - Collaborators Column'))

# #        i.	Check Collaborators Column
# #         2.	Check no empty or missing names
# #2.	Check no empty (None, N/A NaN or missing names) 
# cols = rawIraw.columns 
# collaborators = rawIraw[[rawIraw.columns[1]]].copy()
# df = collaborators.copy()
# isnullDF = df.replace(r'^\s*$', np.nan, regex=True).isnull()
# numMissingTeams = isnullDF.iloc[:,0].sum()


# def click_on_show_intx_cbmissing(click):
#     clear_output()
#     display(intxcbmissing_mkdn)
#     display(hideintx_cb_missing)
    
#     print("numMissingTeams Names: ", numMissingTeams)
#     if(numMissingTeams>0):
#         errorMessage = "Team name is mandatory. \"Team\" column cannot be empty or blank." 
#         emptyRowList = (np.where(isnullDF.iloc[:,0])[0]).tolist()

# showintx_cb_missing.on_click(click_on_show_intx_cbmissing)
# hideintx_cb_missing.on_click(hide_details_intx_cbmissing)          
        
# display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

In [ ]:

clear_output()

showintx_uniquenames = widgets.Button(description ='Show details')
hideintx_uniquenames = widgets.Button(description ='Hide details')
intxuniquenames_mkdn = widgets.Output()
with intxuniquenames_mkdn:
    display(Markdown('<b> Get unique name of Teams</b>'))
display(intxuniquenames_mkdn)
pass_or_fail_mkdwn15 = widgets.Output()
with pass_or_fail_mkdwn15:
    clear_output()
    display(HTML('<p>&#9989;'))

display(widgets.HBox([pass_or_fail_mkdwn15,showintx_uniquenames]))

display(out)


def hide_details_intx_uniquenames(click):
    clear_output()
    display(intxuniquenames_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn15,showintx_uniquenames]))
    display(out)
    
#display(Markdown('#####  Get unique name of Teams'))


# convert team names to string for both TI & CB cols:
rawIraw[Col_1] = rawIraw[Col_1].astype(str)
rawIraw[Col_2] = rawIraw[Col_2].astype(str)

teamsOnCol1 = set(rawIraw[rawIraw.columns[0]])
teamsOnCol2 = set(rawIraw[rawIraw.columns[1]])
allNames = set(teamsOnCol1).union(set(teamsOnCol2))


def click_on_show_intx_uniquenames(click):
    clear_output()
    display(intxuniquenames_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn15,hideintx_uniquenames]))

    print("Number of Raw Team Names in " + Col_1 +": ", len(teamsOnCol1))
    print("Number of Raw Team Names in " + Col_2 +": ", len(teamsOnCol2))
    print("Number or Raw Team Names in Both Cols: ", len(allNames))
    display(out)
    
showintx_uniquenames.on_click(click_on_show_intx_uniquenames)
hideintx_uniquenames.on_click(hide_details_intx_uniquenames)    

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [87]:

clear_output()

showintx_rmvnames = widgets.Button(description ='Show details')
hideintx_rmvnames = widgets.Button(description ='Hide details')
rmvnames_mkdn = widgets.Output()
with rmvnames_mkdn:
    display(Markdown('<b> Acceptable names but will be removed -</b> ["Other_Collaborators", "Unclassified_Internal", "Unclassified_External", "Collaborators Within Group"]'))
display(rmvnames_mkdn)

pass_or_fail_mkdwn16 = widgets.Output()
with pass_or_fail_mkdwn16:
    clear_output()
    display(HTML('<p>&#9989;'))

display(widgets.HBox([pass_or_fail_mkdwn16,showintx_rmvnames]))

display(out)


def hide_details_intx_rmvnames(click):
    clear_output()
    display(rmvnames_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn16,showintx_rmvnames]))
    display(out)

# Check WPA specific names: For Example
#      a.	These are acceptable names but will be removed (not counted) =
#       ["Other_Collaborators", "Unclassified_Internal", "Unclassified_External", "Collaborators Within Group"]
removeNames = ["Other_Collaborators", "Unclassified_Internal", "Unclassified_External", "Collaborators Within Group"]
keepNames = allNames - set(removeNames)

def click_on_show_intx_rmvnames(click):
    clear_output()
    display(rmvnames_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn16,hideintx_rmvnames]))
    print("Number of Raw Team Names in Both less WPA Removed: ", len(keepNames))
    display(out)

showintx_rmvnames.on_click(click_on_show_intx_rmvnames)
hideintx_rmvnames.on_click(hide_details_intx_rmvnames)    

    
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

Output()

Output(layout=Layout(border='1px solid black'))

<IPython.core.display.Javascript object>

In [88]:

clear_output()

temp1 = rawIraw.copy()
cols = rawIraw.columns
temp2 = temp1.loc[temp1[cols[1]].astype(str) =='Collaborators Within Group', cols[1]]
temp3 = temp1.loc[temp1[cols[0]].astype(str) =='Collaborators Within Group', cols[0]]

showintx_cbacceptable = widgets.Button(description ='Show details')
hideintx_cbacceptable= widgets.Button(description ='Hide details')
intx_acptcb_mkdn = widgets.Output()
with intx_acptcb_mkdn:
    display(Markdown('<b> "Collaborators Within Group” is acceptable in Collaborators column ONLY </b>'))
display(intx_acptcb_mkdn)


pass_or_fail_mkdwn17 = widgets.Output()
with pass_or_fail_mkdwn17:
    clear_output()
    if (len(temp3) == 0 and len(temp2) >= 0):
        display(HTML('<p>&#9989;'))
    else:
        display(HTML('&#10060;'))

display(widgets.HBox([pass_or_fail_mkdwn17,showintx_cbacceptable]))
 
display(out)

#display(Markdown('####  "Collaborators Within Group” is acceptable in Collaborators column ONLY'))

def hide_details_intx_cbacpt(click):
    clear_output()
    display(intx_acptcb_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn17,showintx_cbacceptable]))
    display(out)



def click_on_show_intx_cbacpt(click):
    clear_output()
    display(intx_acptcb_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn17,hideintx_cbacceptable]))
    print("Number of Collaborators_Within_Group tag in Time Investors Col: ", len(temp3))
    print("Number of Collaborators_Within_Group tag in Collaborators Col: ", len(temp2))
    if (len(temp3) == 0 and len(temp2) >= 0):
        print("Collaborators_within_group tags are OK")
    else: 
        print("collaborators_within_group tags are INVALID")
    display(out)

        
showintx_cbacceptable.on_click(click_on_show_intx_cbacpt)
hideintx_cbacceptable.on_click(hide_details_intx_cbacpt)        
        
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))    

Output()

Output(layout=Layout(border='1px solid black'))

<IPython.core.display.Javascript object>

In [89]:

clear_output()


uniqueTeams1 = teamsOnCol1 - set(removeNames)
uniqueTeams2 = teamsOnCol2 - set(removeNames)
if(len(uniqueTeams2) > len(uniqueTeams1)):
    uniqueDiff = uniqueTeams2 - uniqueTeams1
else:
    uniqueDiff = uniqueTeams1 - uniqueTeams2
    
    
showintx_namesmatch = widgets.Button(description ='Show details')
hideintx_namesmatch= widgets.Button(description ='Hide details')
intx_namesmatch_mkdn = widgets.Output()
with intx_namesmatch_mkdn:
    display(Markdown('<b> Ensure Time Collaborators + Time Investors columns names match </b>'))
display(intx_namesmatch_mkdn)

pass_or_fail_mkdwn18 = widgets.Output()
with pass_or_fail_mkdwn18:
    clear_output()
    if(len(uniqueTeams1) != len(uniqueTeams2)): 
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))

display(widgets.HBox([pass_or_fail_mkdwn18,showintx_namesmatch]))
 
display(out)
#display(Markdown('####  Ensure Time Collaborators + Time Investors columns names match'))



def hide_details_intx_namesmatch(click):
    clear_output()
    display(intx_namesmatch_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn18,showintx_namesmatch]))
    display(out)

            
def click_on_show_intx_namesmatch(click):
    clear_output()
    display(intx_namesmatch_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn18,hideintx_namesmatch]))
    print("Number of unique Teams in " + Col_1 +":", len(uniqueTeams1))
    print("Number of unique Teams in " + Col_2 +":", len(uniqueTeams2))
    if(len(uniqueTeams1) != len(uniqueTeams2)): 
        print("ERROR: Time Collaborators + Time Investors names don't match")
        print("Teams that don't match... ")

    display(uniqueDiff)
    display(out)

showintx_namesmatch.on_click(click_on_show_intx_namesmatch)
hideintx_namesmatch.on_click(hide_details_intx_namesmatch)  
            
            
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

Output()

Output(layout=Layout(border='1px solid black'))

<IPython.core.display.Javascript object>

In [90]:

clear_output()

intFileNames = keepNames
teamSizeFileNames = set(teamCatalog.iloc[:,0])


showintx_namesmatchwtsz = widgets.Button(description ='Show details')
hideintx_namesmatchwtsz= widgets.Button(description ='Hide details')
intx_namesmatchwtsz_mkdn = widgets.Output()
with intx_namesmatchwtsz_mkdn:
    display(Markdown('<b> Ensure Time Collaborators + Time Investors columns names match --> with Team Size File </b>'))
display(intx_namesmatchwtsz_mkdn)

pass_or_fail_mkdwn19 = widgets.Output()
with pass_or_fail_mkdwn19:
    clear_output()
    if(len(teamSizeFileNames) != len(intFileNames)):
        display(HTML('&#10060;'))
    else:
        display(HTML('<p>&#9989;'))

display(widgets.HBox([pass_or_fail_mkdwn19,showintx_namesmatchwtsz]))

#display(Markdown('####  Ensure Time Collaborators + Time Investors columns names match --> with Team Size File'))
display(out)

def hide_details_intx_namesmatchwtsz(click):
    clear_output()
    display(intx_namesmatchwtsz_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn19,showintx_namesmatchwtsz]))  
    display(out)



def click_on_show_intx_namesmatchwtsz(click):
    clear_output()
    display(intx_namesmatchwtsz_mkdn)
    display(widgets.HBox([pass_or_fail_mkdwn19,hideintx_namesmatchwtsz])) 
    display(out)

    print("Number of Teams in Team Size File: ", len(teamSizeFileNames))
    print("Number of Teams in Interactions File: ", len(intFileNames))
    if(len(teamSizeFileNames) == len(intFileNames)):
        print("The total number of teams in both files match")
        fileNamesDiff = teamSizeFileNames ^ intFileNames
    elif(len(teamSizeFileNames) > len(intFileNames)):
        fileNamesDiff = teamSizeFileNames - intFileNames
    else:
        fileNamesDiff = intFileNames - teamSizeFileNames 
    display(fileNamesDiff)
    
    
showintx_namesmatchwtsz.on_click(click_on_show_intx_namesmatchwtsz)
hideintx_namesmatchwtsz.on_click(hide_details_intx_namesmatchwtsz)  

display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

Output()

Output(layout=Layout(border='1px solid black'))

<IPython.core.display.Javascript object>

In [91]:

clear_output()

scenario2_markdown = widgets.Output()

with scenario2_markdown:
    display(Markdown('<div class="alert alert-block alert-warning"><b> Scenario 2: </b> If teams missing from Team size file but existing in one or both of the Time Investor/Time Collaborator columns, prompt user with message that “Team name exists in Interaction File that does not exist in Team Size file: {name1, name2, etc.}” and stop validation.</div>'))

display(scenario2_markdown)


#Report differences - Across all teams files and columns
diffTS1 = teamSizeFileNames - uniqueTeams1
diffTS2 = teamSizeFileNames - uniqueTeams2

diffIn1 = uniqueTeams1 - teamSizeFileNames
diffIn2 = uniqueTeams2 - teamSizeFileNames

diffIn3 = uniqueTeams1 - uniqueTeams2
diffIn4 = uniqueTeams2 - uniqueTeams1
diffIn5 = uniqueTeams1 ^ uniqueTeams2


    
    

if len(diffTS1)> 0:
    print("Teams in Team Size and NOT in Interactions " + Col_1 +"... ")
    print("     ", diffTS1)
else:
    print("Teams in Team Size and NOT in Interactions " + Col_1 +"... ")
    print("     ", 'None')
    
print("Teams in Team Size and NOT in Interactions " + Col_2 +"... ")
print("     ", diffTS2)


print("Teams in " + Col_1 +" and not in " + Col_2 +"...")
print("     ", diffIn3)
print("Teams in " + Col_2 +" and not in " + Col_1 +"...")
print("     ", diffIn4)
print("Total teams mismatches between " + Col_1 + " and " + Col_2)
print("     ", diffIn5)




if len(diffIn1) > 0 and len(diffIn2) > 0:
    totaldiff1_2 = list(set(diffIn1) | set(diffIn2))
    
elif len(diffIn1) > 0 and len(diffIn2) < 1:
    totaldiff1_2 = list(set(diffIn1))
                        
elif len(diffIn1) < 1 and len(diffIn2) > 0:
    totaldiff1_2 = list(set(diffIn2))

    
    
scenario2error_markdown = widgets.Output()

with scenario2error_markdown:
    display(Markdown('<div class="alert alert-block alert-danger"><b> ERROR!: </b> Team name exists in Interaction File that <b>does not exist</b> in Team Size file: {'+str(totaldiff1_2)+'}.</div>'))
    
    
if len(diffIn1) < 1 and  len(diffIn2) < 1:
    
    print("Teams in " + Col_1 +" and not in Team Size... ")
    print("None")
    print("Teams in " + Col_2 +" and not in Team Size... ")
    print("None") 
    display(HTML('<p>&#9989; Passed</p>'))
    display(out)
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+1)'))
else:
    display(scenario2error_markdown)
    #display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    display(out)
    

    
# date_stamp = ""
# out_dir = "Abel_tests_003" + date_stamp + "/"
# out_dir


Output()

Teams in Team Size and NOT in Interactions TimeInvestors_Organization... 
      None
Teams in Team Size and NOT in Interactions Collaborators_Organization... 
      set()
Teams in TimeInvestors_Organization and not in Collaborators_Organization...
      set()
Teams in Collaborators_Organization and not in TimeInvestors_Organization...
      set()
Total teams mismatches between TimeInvestors_Organization and Collaborators_Organization
      set()
Teams in TimeInvestors_Organization and not in Team Size... 
None
Teams in Collaborators_Organization and not in Team Size... 
None


Output(layout=Layout(border='1px solid black'))

<IPython.core.display.Javascript object>

In [92]:
style = {'description_width': 'initial'}

superoutput_toggle = widgets.ToggleButtons(
    options=['Total Collaboration', 'Boost Intra-Collab', 'Latest date','Avg. Collaboration'],
    disabled=False,
    value=None,
    button_style='primary',
    style = style
    
    
)

boost_slider = widgets.IntSlider(value=0, min=0,max=10,step=1, description = 'Boost Multiplier:', style=style)

boost_ready = widgets.Button(description='Ready',button_style='success')


scenario3_markdown = widgets.Output()


with scenario3_markdown:
    display(Markdown('<div class="alert alert-block alert-warning"><b> Scenario 3: </b> Team name is in Team size file and only showing for the TimeInvestor or Collaborator column within Interaction file. Missing combinations of collaboration among these teams will be generated here... </div>'))

    
display(scenario3_markdown)
#display(superoutput_toggle)
        
pick_output_markdown = widgets.Output()


with pick_output_markdown:
    display(Markdown('<div class="alert alert-block alert-info"> Which output scenario do you need: <b>Latest date</b>,<b> Average Collaboration</b>, <b>Total Collaboration</b>, or <b>Boost Intra-Collaboration</b>?</div>'))
    display(Markdown("<b>Total Collaboration</b>: This will group by teams( both Investor and Collaborator) and get total(sum) hours spent."))    
    display(Markdown("<b>Latest date</b>: This will pick the latest interaction date from a Team to Team collaboration."))
    display(Markdown("<b>Average Collaboration</b>: This will group by teams( both Investor and Collaborator) and get average hours spent."))
    display(Markdown("<b>Boost Intra-Collab</b>: This will boost collaboration amongst same teams( both Investor and Collaborator)."))    
    
if len(diffIn1) < 1 and  len(diffIn2) < 1:
    clear_output()
    display(pick_output_markdown)
    display(superoutput_toggle)
else:
    clear_output()


def boost_multiplier_on_change(change):
    global boostv 
    boostv = change.new
    
    
    
def ready_clicked(clicked):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))





def toggle_on_click_output(change):
    if change.new =='Latest date':
        clear_output()
        display(scenario3_markdown)
        display(widgets.VBox([pick_output_markdown,superoutput_toggle]))
        display(Markdown("<b>Latest date</b>: This will pick the latest interaction date from a Team to Team collaboration."))
        display(out)
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    
    elif change.new=='Avg. Collaboration':
        clear_output()
        display(scenario3_markdown)
        display(widgets.VBox([pick_output_markdown,superoutput_toggle]))
        display(Markdown("<b>Average Collaboration</b>: This will group by teams( both Investor and Collaborator) and get average hours spent."))
        display(out)
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    
    elif change.new=='Total Collaboration':
        clear_output()
        display(scenario3_markdown)
        display(widgets.VBox([pick_output_markdown,superoutput_toggle]))
        display(Markdown("<b>Total Collaboration</b>: This will group by teams( both Investor and Collaborator) and get total(sum) hours spent."))
        display(out)
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))
    
    elif change.new=='Boost Intra-Collab':
        clear_output()
        display(scenario3_markdown)
        display(widgets.VBox([pick_output_markdown,superoutput_toggle]))
        display(Markdown("<b>Boost Intra-Collab</b>: This will boost collaboration amongst same teams( both Investor and Collaborator)."))
        display(widgets.HBox([boost_slider,boost_ready]))
        display(out)
superoutput_toggle.observe(toggle_on_click_output, 'value')
boost_slider.observe(boost_multiplier_on_change,names='value')
boost_ready.on_click(ready_clicked)

Output()

ToggleButtons(button_style='primary', options=('Total Collaboration', 'Boost Intra-Collab', 'Latest date', 'Av…

In [93]:
if len(diffIn1) < 1 and  len(diffIn2) < 1:

    names = list(rawIraw.columns[0:2])

    tsnames = list(set(rawTS['Team']))
    tsnames2 = list(set(rawTS['Team']))

    combined = [tsnames, tsnames2]
    today = date.today()
    datetoday = today.strftime("%m/%d/%y")


    if superoutput_toggle.value == 'Latest date':   
        mergenames = list(names)
        mergenames.append('Date')


        rawIraw['Date'] = pd.to_datetime(rawIraw['Date'], format='%m/%d/%Y')

        data_last_date = rawIraw.groupby(names, as_index=False).agg({"Date": "max"})


        data_1 = pd.merge(data_last_date, rawIraw,on=mergenames,  how='inner')

        #ti = data_1.columns[0]
        #cb = data_1.columns[1]
        #unique_gg = pd.concat([data_1[ti].rename(columns={ti: 'team'}, inplace=True),data_1[cb].rename(columns={cb: 'team'}, inplace=True)]).drop_duplicates()


        #unique_TI= data_1[ti].rename(columns={ti: 'team'}, inplace=True).drop_duplicates()
        #unique_CB = data_1[cb].rename(columns={cb: 'team'}, inplace=True).drop_duplicates()



        #d = []
        #for x in unique_TI:
            #for y in unique_CB:
                #d.append((x,y))

        #allComb = pd.DataFrame(d,columns=names)
        missing_combinationsDF = pd.DataFrame(columns = names, data=list(itertools.product(*combined)))
        missing_combinationsDF = missing_combinationsDF.astype(str)

        superinteractionsDF = pd.merge(missing_combinationsDF, data_1, on=names, how= "outer").fillna(0)
        missingvaluereportDF = superinteractionsDF[superinteractionsDF['Collaboration_hours']==0]
        numofmissingrecords = missingvaluereportDF.shape[0]




    elif superoutput_toggle.value == 'Avg. Collaboration': 

        missing_combinationsDF = pd.DataFrame(columns = names, data=list(itertools.product(*combined)))
        missing_combinationsDF = missing_combinationsDF.astype(str)

        superinteractionsDF = missing_combinationsDF.merge(rawIraw, how='outer').fillna(0)
        superinteractionsDF.Date.replace([0],datetoday,inplace=True)

        rawagg_interactionsDF = rawIraw.groupby(names).sum().reset_index()

        missingvaluereportDF = superinteractionsDF[superinteractionsDF['Collaboration_hours']==0]

        superinteractionsDF = superinteractionsDF.groupby(names).mean().reset_index()

        superinteractionsDF['Date']=datetoday
        col_order = list(names)
        col_order.append('Date')
        col_order.append('Collaboration_hours')


        superinteractionsDF = superinteractionsDF.reindex(columns=col_order)

        numofmissingrecords = missingvaluereportDF.shape[0]


    elif superoutput_toggle.value == 'Total Collaboration': 

        missing_combinationsDF = pd.DataFrame(columns = names, data=list(itertools.product(*combined)))
        missing_combinationsDF = missing_combinationsDF.astype(str)

        superinteractionsDF = missing_combinationsDF.merge(rawIraw, how='outer').fillna(0)
        superinteractionsDF.Date.replace([0],datetoday,inplace=True)

        rawagg_interactionsDF = rawIraw.groupby(names).sum().reset_index()

        missingvaluereportDF = superinteractionsDF[superinteractionsDF['Collaboration_hours']==0]

        superinteractionsDF = superinteractionsDF.groupby(names).sum().reset_index()

        superinteractionsDF['Date']=datetoday
        col_order = list(names)
        col_order.append('Date')
        col_order.append('Collaboration_hours')


        superinteractionsDF = superinteractionsDF.reindex(columns=col_order)


        numofmissingrecords = missingvaluereportDF.shape[0]
        
    elif superoutput_toggle.value == 'Boost Intra-Collab': 
        
        output_slider = widgets.Output()
        with output_slider:
            print("Collaboration will boost using "+ str(boostv)+"X multiplier of max collaboration")
        display(output_slider)

        missing_combinationsDF = pd.DataFrame(columns = names, data=list(itertools.product(*combined)))
        missing_combinationsDF = missing_combinationsDF.astype(str)

        superinteractionsDF = missing_combinationsDF.merge(rawIraw, how='outer').fillna(0)
        superinteractionsDF.Date.replace([0],datetoday,inplace=True)

        rawagg_interactionsDF = rawIraw.groupby(names).sum().reset_index()

        missingvaluereportDF = superinteractionsDF[superinteractionsDF['Collaboration_hours']==0]

        superinteractionsDF = superinteractionsDF.groupby(names).sum().reset_index()

        maxcolumn = superinteractionsDF["Collaboration_hours"]
        max_value = maxcolumn.max() 
        boosted_value = max_value * boostv

        superinteractionsDF['Collaboration_hours'] = np.where(superinteractionsDF[names[0]] == superinteractionsDF[names[1]], boosted_value, superinteractionsDF['Collaboration_hours'])

        superinteractionsDF['Date']=datetoday
        col_order = list(names)
        col_order.append('Date')
        col_order.append('Collaboration_hours')


        superinteractionsDF = superinteractionsDF.reindex(columns=col_order)


        numofmissingrecords = missingvaluereportDF.shape[0]


    if superoutput_toggle.value == 'Latest date':

        display(Markdown("Total records in original Interaction file : " + "<b>"+str(rawIraw.shape[0])+"</b>"))
        display(Markdown("Total records with reduced  latest date Interaction file : " + "<b>"+str(data_1.shape[0])+"</b>"))
        display(Markdown("Total records added based on missing combinations : " + "<b>"+str(numofmissingrecords)+"</b>"))
        display(Markdown('Total records for new  "Superinteractions"  file : '+ "<b>"+str(numofmissingrecords+data_1.shape[0]) +"</b>"))
        display(HTML('<p>&#9989; Done </p>'))
        display(out)
        
    elif superoutput_toggle.value == 'Avg. Collaboration':

        display(Markdown("Total records in original Interaction file : " + "<b>"+str(rawIraw.shape[0])+"</b>"))
        display(Markdown("Total records with reduced by agg{<b>average</b>} Interaction file : " + "<b>"+str(rawagg_interactionsDF.shape[0])+"</b>"))  
        display(Markdown("Total records added based on missing combinations : " + "<b>"+str(numofmissingrecords)+"</b>"))
        display(Markdown('Total records for new  "Superinteractions"  file : '+ "<b>"+str(numofmissingrecords+rawagg_interactionsDF.shape[0]) +"</b>"))
        display(HTML('<p>&#9989; Done </p>'))
        display(out)

    elif superoutput_toggle.value == 'Total Collaboration':

        display(Markdown("Total records in original Interaction file : " + "<b>"+str(rawIraw.shape[0])+"</b>"))
        display(Markdown("Total records with reduced by agg{<b>sum</b>} Interaction file : " + "<b>"+str(rawagg_interactionsDF.shape[0])+"</b>"))  
        display(Markdown("Total records added based on missing combinations : " + "<b>"+str(numofmissingrecords)+"</b>"))
        display(Markdown('Total records for new  "Superinteractions"  file : '+ "<b>"+str(numofmissingrecords+rawagg_interactionsDF.shape[0]) +"</b>"))
        display(HTML('<p>&#9989; Done </p>'))
        display(out)
        
    elif superoutput_toggle.value == 'Boost Intra-Collab':

        display(Markdown("Total records in original Interaction file : " + "<b>"+str(rawIraw.shape[0])+"</b>"))
        display(Markdown("Total records with reduced by agg{<b>sum</b>} Interaction file : " + "<b>"+str(rawagg_interactionsDF.shape[0])+"</b>")) 
        display(Markdown("Hours boosted value for Group A matching Group B : " + "<b>"+str(boosted_value)+"</b>"))
        display(Markdown("Total records added based on missing combinations : " + "<b>"+str(numofmissingrecords)+"</b>"))
        display(Markdown('Total records for new  "Superinteractions"  file : '+ "<b>"+str(numofmissingrecords+rawagg_interactionsDF.shape[0]) +"</b>"))
        display(HTML('<p>&#9989; Done </p>'))
        display(out)
else:
    clear_output()
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [94]:
if len(diffIn1) < 1 and  len(diffIn2) < 1:

    keyCol_1 = missingvaluereportDF.columns[0]
    keyCol_2 = missingvaluereportDF.columns[1]

    mvrMatrixDF = missingvaluereportDF.pivot_table(index=keyCol_1, columns=keyCol_2,values="Collaboration_hours")

    mvrMatrixDF.columns.name = None
    mvrMatrixDF.index.name = None
    mvrMatrixDF = mvrMatrixDF.reset_index()
    mvrMatrixDF = mvrMatrixDF.fillna('Not Missing')
    mvrMatrixDF.drop(mvrMatrixDF.columns[0],axis=1)
    mvrMatrixDF.rename(columns = {mvrMatrixDF.columns[0]: 'Collab' }, inplace = True)
else:
    clear_output()
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [95]:
if len(diffIn1) < 1 and  len(diffIn2) < 1:
    def color_missing(val):
        color = 'red' if val == 0 else 'black'
        #color = 'white' if val == 'Not Missing' else 'black'
        return 'color: %s' % color
    if numofmissingrecords > 0:
        display(Markdown('Missing values matrix shown below: <b> "Not Missing" </b> means an existing collaboration exists'))
        display(Markdown('Please wait for output...'))
        display(mvrMatrixDF.style.applymap(color_missing))
        display(out)
    else:
        clear_output()
else:
    clear_output()
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [96]:
root = './Final Files set/'
if len(diffIn1) < 1 and  len(diffIn2) < 1:

    try:
        if not (os.path.exists(root) and os.path.isdir(root)):
            os.mkdir(root)
            print ("Successfully created the directory %s " % root)
    except OSError:
        print ("Creation of the directory %s failed" % root)

    superinteractionsDF.to_csv(root+'interactions.csv',index = False)


    rawTS.to_csv(root+'team_size.csv',index = False)
    rawFC.to_csv(root+'space_capacity.csv',index = False)
    rawDT.to_csv(root+'distance.csv',index = False)

    mvrMatrixDF.to_csv('missingvaluesMatrix.csv',index = False)

    saveintxfinsh = '<div class="alert alert-block alert-success"><b> Done: </b> Your "super" interaction file <b>(interactions.csv)</b> along with the other 3 files(<b>team_size.csv, space_capacity.csv, and distance.csv </b>) have successfully saved back to a new sub-directory inside the location of this Notebook! Folder name is [<b>Final Files set</b>].</div>'

else:
    clear_output()
    
display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

<IPython.core.display.Javascript object>

In [97]:
if len(diffIn1) < 1 and  len(diffIn2) < 1:
    display(Markdown(saveintxfinsh))
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

else:
    clear_output()


<div class="alert alert-block alert-success"><b> Done: </b> Your "super" interaction file <b>(interactions.csv)</b> along with the other 3 files(<b>team_size.csv, space_capacity.csv, and distance.csv </b>) have successfully saved back to a new sub-directory inside the location of this Notebook! Folder name is [<b>Final Files set</b>].</div>

<IPython.core.display.Javascript object>

In [98]:
style2 = {'description_width': '425px'}
layout2 = {'width': '300px'}
startAlgorithmNB = widgets.Button(description ='Open Generate Floorplan Notebook',style=style2,layout=layout2,button_style='info')
display(startAlgorithmNB)

def go_to_nextblock(btn):
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)'))

startAlgorithmNB.on_click(go_to_nextblock)           
        

Button(button_style='info', description='Open Generate Floorplan Notebook', layout=Layout(width='300px'), styl…

In [99]:
%%javascript
var name_of_the_notebook = 'WPP.Generate-Floorplan_PIPE-003_v2.1.1.ipynb'
var url = window.location.href.split('/')
var newurl = url[0] + '//'
for (var i = 1; i < url.length - 1; i++) {
    console.log(url[i], newurl)
    newurl += url[i] + '/'
}
newurl += name_of_the_notebook
window.open(newurl)

<IPython.core.display.Javascript object>

In [100]:
# melted_MatrixDF = mvrMatrixDF.melt(id_vars=['Collab'], var_name = ['Teams'])
# only_missingDF = melted_MatrixDF[melted_MatrixDF['value']==0]


In [101]:
# display(widgets.HTML(value="<ins><b>Interaction file</b></ins>"))

# print(rawIraw.isnull().sum())

# display(widgets.HTML(value="<ins><b>Team size file</b></ins>"))
# print(rawTS.isnull().sum())

# display(widgets.HTML(value="<ins><b>Floor capacity file</b></ins>"))
# print(rawFC.isnull().sum())


# display(widgets.HTML(value="<ins><b>Distance file</b></ins>"))
# print(rawDT.isnull().sum())